<a href="https://colab.research.google.com/github/AnamariaVLR/noura-rag/blob/main/NOURA_RAG_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# NOURA - Cell 1: Verify environment
print("NOURA is starting...")
print("Python ready")

NOURA is starting...
Python ready


In [5]:
# NOURA - Cell 2: Scoring methodology (NOURA core IP)

EVIDENCE_HIERARCHY = {
    "systematic_review_meta_analysis": {"base_weight": 1.00, "requires_independence_check": True},
    "rct":                             {"base_weight": 0.85, "requires_independence_check": True},
    "regulatory_opinion":              {"base_weight": 0.75, "requires_independence_check": False},
    "observational_cohort":            {"base_weight": 0.60, "requires_independence_check": True},
    "ewg_hazard":                      {"base_weight": 0.50, "requires_dose_adjustment": True},
    "cosing_regional":                 {"base_weight": 0.45, "requires_independence_check": False},
    "in_vitro":                        {"base_weight": 0.30, "requires_independence_check": False},
    "clinical_case":                   {"base_weight": 0.15, "requires_independence_check": False},
    "expert_opinion":                  {"base_weight": 0.10, "requires_independence_check": True},
}

INDUSTRY_FUNDING_PENALTY = 0.20
HEALTH_HARD_BLOCK = 50
PLANET_FLAG_THRESHOLD = 40

SUFFICIENCY_CAPS = {
    "only_in_vitro_or_case":   60,
    "only_regulatory_strong":  70,
    "only_regulatory_weak":    50,
    "single_rct":              80,
}

CATEGORY_CLAIM_REQUIREMENTS = {
    "skincare": {
        "hydration":   {"min_evidence": "rct", "min_studies": 1},
        "anti_aging":  {"min_evidence": "rct", "min_studies": 2},
        "brightening": {"min_evidence": "observational_cohort", "min_studies": 1},
        "acne":        {"min_evidence": "rct", "min_studies": 2},
    }
}

print("Scoring methodology loaded")
print(f"Evidence sources defined: {len(EVIDENCE_HIERARCHY)}")
print(f"Health hard block threshold: {HEALTH_HARD_BLOCK}")

Scoring methodology loaded
Evidence sources defined: 9
Health hard block threshold: 50


In [6]:
# NOURA - Cell 3: Scoring engine

def evaluate_evidence(source_type, industry_funded=False, dose_adjusted=True):
    if source_type not in EVIDENCE_HIERARCHY:
        return {"weight": 0, "source_type": source_type, "flags": [f"Unknown source type: {source_type}"]}

    weight = EVIDENCE_HIERARCHY[source_type]["base_weight"]
    flags = []

    if industry_funded and EVIDENCE_HIERARCHY[source_type].get("requires_independence_check"):
        weight = weight * (1 - INDUSTRY_FUNDING_PENALTY)
        flags.append("Industry-funded study: weight reduced 20%")

    if source_type == "ewg_hazard" and not dose_adjusted:
        weight = 0
        flags.append("EWG score excluded: not adjusted for actual product concentration")

    return {"source_type": source_type, "weight": round(weight, 3), "flags": flags}


def calculate_health_score(evaluated_evidence, prohibited=False):
    if prohibited:
        return {
            "score": 0,
            "verdict": "HIGHER RISK",
            "flag": "Ingredient prohibited by regulatory authority",
            "evidence_situation": "regulatory_block"
        }

    if not evaluated_evidence:
        return {
            "score": None,
            "verdict": "INSUFFICIENT DATA",
            "flag": "No scientific evidence retrieved for this ingredient",
            "evidence_situation": "no_evidence"
        }

    source_types = [e["source_type"] for e in evaluated_evidence]
    only_lab = all(t in {"in_vitro", "clinical_case"} for t in source_types)
    only_regulatory = all(t in {"regulatory_opinion", "cosing_regional"} for t in source_types)

    # FIXED FORMULA: use top 3 highest-weight studies, not average of all
    # This prevents weak studies from dragging down a strong evidence base
    weights = sorted([e["weight"] for e in evaluated_evidence], reverse=True)
    top_weights = weights[:3]
    score_raw = round((sum(top_weights) / len(top_weights)) * 100, 1)

    # Apply sufficiency caps
    if only_lab:
        score = min(score_raw, SUFFICIENCY_CAPS["only_in_vitro_or_case"])
        flag = "Health score capped at 60 — only lab-based evidence retrieved; human clinical data insufficient"
        situation = "only_lab"
    elif only_regulatory:
        score = min(score_raw, SUFFICIENCY_CAPS["only_regulatory_strong"])
        flag = "Health score capped at 70 — regulatory approval present but no clinical studies retrieved"
        situation = "only_regulatory"
    else:
        score = score_raw
        flag = None
        situation = "sufficient"

    # Assign verdict
    if score is None or score < HEALTH_HARD_BLOCK:
        verdict = "HIGHER RISK"
    elif score >= 71:
        verdict = "WELL SUPPORTED"
    elif score >= 41:
        verdict = "LIMITED SUPPORT"
    else:
        verdict = "HIGHER RISK"

    return {
        "score": score,
        "verdict": verdict,
        "flag": flag,
        "evidence_situation": situation
    }

print("Scoring engine loaded")

Scoring engine loaded


In [7]:
# NOURA - Cell 4: PubMed connection with abstract retrieval (50 studies)
import requests
import time
import xml.etree.ElementTree as ET
import re

def search_pubmed(ingredient, max_results=50):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"

    time.sleep(1)

    # Step 1: Search for IDs
    search_response = requests.get(
        f"{base_url}esearch.fcgi",
        params={
            "db": "pubmed",
            "term": ingredient + "[Title]",
            "retmax": max_results,
            "retmode": "json",
            "sort": "relevance"
        }
    )

    search_data = search_response.json()

    if "esearchresult" not in search_data:
        return {"ingredient": ingredient, "studies_found": 0, "studies": [],
                "error": "PubMed rate limit — try again in 30 seconds"}

    ids = search_data["esearchresult"]["idlist"]

    if not ids:
        return {"ingredient": ingredient, "studies_found": 0, "studies": []}

    time.sleep(1)

    # Step 2: Fetch abstracts via XML
    fetch_response = requests.get(
        f"{base_url}efetch.fcgi",
        params={
            "db": "pubmed",
            "id": ",".join(ids),
            "retmode": "xml",
            "rettype": "abstract"
        }
    )

    time.sleep(1)

    # Step 3: Fetch metadata
    summary_response = requests.get(
        f"{base_url}esummary.fcgi",
        params={"db": "pubmed", "id": ",".join(ids), "retmode": "json"}
    )
    summary_data = summary_response.json()

    if "result" not in summary_data:
        return {"ingredient": ingredient, "studies_found": 0, "studies": []}

    # Step 4: Parse abstracts and sample sizes from XML
    abstracts = {}
    sample_sizes = {}

    try:
        root = ET.fromstring(fetch_response.content)
        for article in root.findall(".//PubmedArticle"):
            pmid_el = article.find(".//PMID")
            if pmid_el is None:
                continue
            pmid = pmid_el.text

            abstract_texts = article.findall(".//AbstractText")
            if abstract_texts:
                abstract = " ".join([el.text or "" for el in abstract_texts])
                abstracts[pmid] = abstract[:600]

            full_text = " ".join([el.text or "" for el in article.findall(".//AbstractText")])
            size_matches = re.findall(
                r'\b(\d+)\s*(?:patients|participants|subjects|women|men|volunteers|individuals|adults)',
                full_text, re.IGNORECASE
            )
            if size_matches:
                sample_sizes[pmid] = max([int(x) for x in size_matches])
    except Exception:
        pass

    # Step 5: Build studies list
    studies = []
    for uid in ids:
        article = summary_data["result"].get(uid, {})
        if isinstance(article, dict) and "title" in article:
            studies.append({
                "id": uid,
                "title": article.get("title", ""),
                "abstract": abstracts.get(uid, ""),
                "sample_size": sample_sizes.get(uid, None),
                "year": article.get("pubdate", "")[:4],
                "source": "PubMed",
                "pubmed_url": f"https://pubmed.ncbi.nlm.nih.gov/{uid}/"
            })

    return {"ingredient": ingredient, "studies_found": len(studies), "studies": studies}

print("PubMed connection ready — up to 50 studies with abstracts")

PubMed connection ready — up to 50 studies with abstracts


In [8]:
# NOURA - Cell 5: Evidence classifier

def classify_evidence_type(title):
    title_lower = title.lower()

    if any(w in title_lower for w in ["meta-analysis", "systematic review", "cochrane"]):
        return "systematic_review_meta_analysis"

    elif any(w in title_lower for w in ["randomized", "rct", "controlled trial", "double-blind", "clinical trial"]):
        return "rct"

    elif any(w in title_lower for w in ["cohort", "observational", "prospective", "retrospective", "epidemiolog"]):
        return "observational_cohort"

    elif any(w in title_lower for w in ["guideline", "regulatory", "safety assessment", "efsa", "fda", "final report"]):
        return "regulatory_opinion"

    elif any(w in title_lower for w in ["review", "overview", "narrative", "update", "current evidence", "mechanisms of action", "mechanistic", "applications of"]):
        return "observational_cohort"  # Reviews treated as observational — higher than in_vitro

    elif any(w in title_lower for w in ["in vitro", "cell culture", "in-vitro"]):
        return "in_vitro"

    elif any(w in title_lower for w in ["case report", "case study"]):
        return "clinical_case"

    else:
        return "in_vitro"

print("Evidence classifier loaded")

Evidence classifier loaded


In [9]:
# NOURA - Cell 6: Full evaluation pipeline

def noura_evaluate(ingredient, category="skincare"):

    # Step 1: Search PubMed
    pubmed_results = search_pubmed(ingredient, max_results=10)

    # Step 2: Classify and evaluate each study
    evaluated = []
    evidence_count = {}

    for study in pubmed_results.get("studies", []):
        source_type = classify_evidence_type(study["title"])
        ev = evaluate_evidence(source_type)
        ev["study_title"] = study["title"][:80]
        ev["year"] = study["year"]
        evaluated.append(ev)
        evidence_count[source_type] = evidence_count.get(source_type, 0) + 1

    # Step 3: Calculate score
    result = calculate_health_score(evaluated)

    # Step 4: Build evidence summary
    evidence_str = " + ".join([f"{v} {k.replace('_', ' ')}"
                                for k, v in evidence_count.items()])

    # Step 5: Display NOURA assessment
    print(f"NOURA Health Assessment: {ingredient.title()} ({category})")
    print("=" * 65)
    print(f"Score:   {result['score']}/100")
    print(f"Verdict: {result['verdict']}")
    print()
    print(f"Studies retrieved:  {pubmed_results['studies_found']} (PubMed)")
    print(f"Evidence types:     {evidence_str if evidence_str else 'None'}")
    print()

    if result['flag']:
        print(f"Note: {result['flag']}")
        print()

    print("Evidence breakdown:")
    for e in evaluated:
        print(f"  [{e['year']}] {e['source_type'].replace('_', ' ')} "
              f"(weight: {e['weight']}) — {e['study_title']}...")

    print()
    print("What would you like next?")
    print("  - View full source links")
    print("  - Compare with alternatives")
    print("  - Check regulatory status")
    print("  - Assess another ingredient")
    print("=" * 65)
    print()

    return result


# Test
noura_evaluate("niacinamide")
noura_evaluate("parabens", category="skincare")

NOURA Health Assessment: Niacinamide (skincare)
Score:   65.0/100
Verdict: LIMITED SUPPORT

Studies retrieved:  10 (PubMed)
Evidence types:     4 observational cohort + 5 in vitro + 1 regulatory opinion

Evidence breakdown:
  [2024] observational cohort (weight: 0.6) — Niacinamide: a review on dermal delivery strategies and clinical evidence....
  [2014] observational cohort (weight: 0.6) — Niacinamide - mechanisms of action and its topical use in dermatology....
  [2005] in vitro (weight: 0.3) — Niacinamide: A B vitamin that improves aging facial skin appearance....
  [2002] in vitro (weight: 0.3) — The effect of niacinamide on reducing cutaneous pigmentation and suppression of ...
  [2021] observational cohort (weight: 0.6) — Cosmeceutical Aptitudes of Niacinamide: A Review....
  [2006] in vitro (weight: 0.3) — The effect of 2% niacinamide on facial sebum production....
  [2024] observational cohort (weight: 0.6) — Mechanistic Insights into the Multiple Functions of Niacinamide: Ther

{'score': 73.3,
 'verdict': 'WELL SUPPORTED',
 'flag': None,
 'evidence_situation': 'sufficient'}

In [14]:
# NOURA - Cell 7: Context-aware evidence direction classifier

def classify_evidence_direction(title, abstract=""):
    full_text = (title + " " + abstract).lower()

    # CONCERN: only match when the ingredient IS the cause of harm
    # These phrases indicate the ingredient causes the problem
    concern_phrases = [
        "linked to cancer", "associated with cancer", "cancer risk",
        "causes cancer", "cancer development",
        "linked to harm", "causes harm", "harmful effects of",
        "toxic effects of", "toxicity of", "hazardous effects",
        "endocrine disrupt", "endocrine-disrupt",
        "carcinogenic", "carcinogen",
        "banned", "restricted use", "prohibited",
        "unsafe for", "adverse effects of",
        "breast cancer", "estrogenic activity",
        "reproductive toxicity", "genotoxic", "mutagenic",
        "significant increase in risk",
        "associated with increased risk",
        "exposure linked", "exposure associated",
        "disrupts hormone", "disrupts endocrine",
        "impairs", "damages skin", "causes damage",
        "causes inflammation", "pro-inflammatory effect",
        "allergic reaction to", "sensitization to",
        "diabesity", "obesogen",
        "harmful", "dangerous to", "prohibited by"
    ]

    # SAFETY: ingredient provides benefit or is confirmed safe
    safety_phrases = [
        "safe", "safety assessment", "well tolerated", "no adverse",
        "no significant adverse", "approved", "permitted", "gras",
        "efficacious", "significant improvement", "effective treatment",
        "beneficial", "protective", "no toxicity observed",
        "no evidence of harm", "clinically proven",
        "significant reduction in", "improvement in skin",
        "recommended", "widely used safely",
        "explored for treatment", "potential treatment",
        "therapeutic application", "used to treat",
        "treatment of", "against cancer", "anti-cancer",
        "skin brightening", "anti-ageing", "anti-aging",
        "skin barrier", "skin care", "dermal benefits",
        "protects", "protection against", "reduces risk",
        "decreased risk", "prevents", "inhibits",
        "well-established", "proven efficacy",
        "moisturizing", "hydrating", "brightening",
        "anti-inflammatory", "antioxidant",
        "disrupts the virus", "disrupts bacterial", "disrupts pathogen"
    ]

    concern_score = sum(1 for w in concern_phrases if w in full_text)
    safety_score = sum(1 for w in safety_phrases if w in full_text)

    if concern_score > safety_score:
        return "CONCERN"
    elif safety_score > concern_score:
        return "SAFETY"
    else:
        return "NEUTRAL"


def classify_evidence_type(title, abstract=""):
    full_text = (title + " " + abstract).lower()

    if any(w in full_text for w in ["meta-analysis", "systematic review", "cochrane"]):
        return "systematic_review_meta_analysis"
    elif any(w in full_text for w in ["randomized", "randomised", "rct",
                                       "controlled trial", "double-blind",
                                       "double blind", "placebo-controlled"]):
        return "rct"
    elif any(w in full_text for w in ["cohort", "prospective", "retrospective",
                                       "epidemiolog", "population-based"]):
        return "observational_cohort"
    elif any(w in full_text for w in ["guideline", "regulatory", "safety assessment",
                                       "efsa", "fda", "final report", "sccs opinion",
                                       "gras", "approved by"]):
        return "regulatory_opinion"
    elif any(w in full_text for w in ["review", "overview", "narrative review",
                                       "current evidence", "mechanisms of action",
                                       "comprehensive review", "literature review"]):
        return "observational_cohort"
    elif any(w in full_text for w in ["in vitro", "cell culture", "in-vitro",
                                       "cell line", "hek293", "keratinocyte"]):
        return "in_vitro"
    elif any(w in full_text for w in ["case report", "case series"]):
        return "clinical_case"
    else:
        return "in_vitro"


print("Context-aware classifier loaded")

Context-aware classifier loaded


In [11]:
# NOURA - Cell 8: Direction-aware scoring engine with sample size weighting

def get_sample_size_multiplier(sample_size):
    """
    Returns a multiplier based on study sample size.
    Larger studies carry more evidential weight.
    """
    if sample_size is None:
        return 1.0       # Unknown size — no adjustment
    elif sample_size >= 1000:
        return 1.5       # Large study — 50% boost
    elif sample_size >= 100:
        return 1.25      # Medium study — 25% boost
    elif sample_size >= 30:
        return 1.0       # Minimum adequate size — no adjustment
    else:
        return 0.75      # Very small study — 25% penalty


def calculate_direction_aware_score(evaluated_evidence, prohibited=False):
    if prohibited:
        return {
            "score": 0,
            "verdict": "HIGHER RISK",
            "flag": "Ingredient prohibited by regulatory authority",
            "evidence_situation": "regulatory_block",
            "concern_count": 0,
            "safety_count": 0,
            "neutral_count": 0
        }

    if not evaluated_evidence:
        return {
            "score": None,
            "verdict": "INSUFFICIENT DATA",
            "flag": "No scientific evidence retrieved for this ingredient",
            "evidence_situation": "no_evidence",
            "concern_count": 0,
            "safety_count": 0,
            "neutral_count": 0
        }

    source_types = [e["source_type"] for e in evaluated_evidence]
    only_lab = all(t in {"in_vitro", "clinical_case"} for t in source_types)
    only_regulatory = all(t in {"regulatory_opinion", "cosing_regional"} for t in source_types)

    safety_weights = []
    concern_weights = []
    neutral_weights = []

    for e in evaluated_evidence:
        direction = e.get("direction") or classify_evidence_direction(
            e["study_title"], e.get("abstract", "")
        )
        # Apply sample size multiplier
        size_multiplier = get_sample_size_multiplier(e.get("sample_size"))
        weight = e["weight"] * size_multiplier

        if direction == "SAFETY":
            safety_weights.append(weight)
        elif direction == "CONCERN":
            concern_weights.append(weight)
        else:
            neutral_weights.append(weight * 0.5)

    safety_count = len(safety_weights)
    concern_count = len(concern_weights)
    neutral_count = len(neutral_weights)

    total_safety = sum(safety_weights) + sum(neutral_weights)
    total_concern = sum(concern_weights)
    net_score = total_safety - (total_concern * 0.5)

    max_possible = sum(e["weight"] for e in evaluated_evidence)
    if max_possible > 0:
        raw_score = round((net_score / max_possible) * 100, 1)
    else:
        raw_score = 0

    raw_score = max(0, min(100, raw_score))

    # Apply sufficiency caps
    if only_lab:
        score = min(raw_score, SUFFICIENCY_CAPS["only_in_vitro_or_case"])
        flag = "Health score capped at 60 — only lab-based evidence retrieved; human clinical data insufficient"
        situation = "only_lab"
    elif only_regulatory:
        score = min(raw_score, SUFFICIENCY_CAPS["only_regulatory_strong"])
        flag = "Health score capped at 70 — regulatory approval present but no clinical studies retrieved"
        situation = "only_regulatory"
    else:
        score = raw_score
        flag = None
        situation = "sufficient"

    # Hard concern override
    if concern_count > safety_count and concern_count >= 3:
        score = min(score, 45)
        flag = f"Majority of retrieved studies raise safety concerns ({concern_count} concern vs {safety_count} safety studies)"

    # Assign verdict
    if score is None or score < HEALTH_HARD_BLOCK:
        verdict = "HIGHER RISK"
    elif score >= 71:
        verdict = "WELL SUPPORTED"
    elif score >= 41:
        verdict = "LIMITED SUPPORT"
    else:
        verdict = "HIGHER RISK"

    return {
        "score": score,
        "verdict": verdict,
        "flag": flag,
        "evidence_situation": situation,
        "concern_count": concern_count,
        "safety_count": safety_count,
        "neutral_count": neutral_count
    }

print("Direction-aware scoring engine with sample size weighting loaded")

Direction-aware scoring engine with sample size weighting loaded


In [16]:
# NOURA - Cell 9: Confidence scoring engine

def calculate_confidence(evaluated_evidence, direction_count, studies_found):
    """
    Calculates how confident NOURA is in its assessment.
    Based on: volume of evidence, evidence quality, direction consistency.

    Returns:
    - confidence_score: 0-100
    - confidence_label: LOW / MODERATE / HIGH / VERY HIGH
    - confidence_notes: list of factors affecting confidence
    """
    notes = []
    score = 0

    # Factor 1: Volume of evidence (max 30 points)
    if studies_found >= 30:
        score += 30
        notes.append(f"Strong evidence base: {studies_found} studies retrieved")
    elif studies_found >= 15:
        score += 20
        notes.append(f"Moderate evidence base: {studies_found} studies retrieved")
    elif studies_found >= 5:
        score += 10
        notes.append(f"Limited evidence base: {studies_found} studies retrieved")
    else:
        score += 0
        notes.append(f"Very limited evidence: only {studies_found} studies retrieved")

    # Factor 2: Evidence quality (max 30 points)
    source_types = [e["source_type"] for e in evaluated_evidence]
    has_systematic_review = "systematic_review_meta_analysis" in source_types
    has_rct = "rct" in source_types
    has_regulatory = "regulatory_opinion" in source_types
    has_observational = "observational_cohort" in source_types

    quality_score = 0
    if has_systematic_review:
        quality_score += 12
        notes.append("Systematic review/meta-analysis present")
    if has_rct:
        quality_score += 10
        notes.append("RCT evidence present")
    if has_regulatory:
        quality_score += 8
        notes.append("Regulatory opinion present")
    if has_observational:
        quality_score += 5
        notes.append("Observational studies present")

    score += min(quality_score, 30)

    # Factor 3: Direction consistency (max 30 points)
    total_directional = direction_count["SAFETY"] + direction_count["CONCERN"]
    if total_directional > 0:
        dominant = max(direction_count["SAFETY"], direction_count["CONCERN"])
        consistency = dominant / total_directional

        if consistency >= 0.85:
            score += 30
            notes.append(f"High directional consistency: {round(consistency*100)}% of studies agree")
        elif consistency >= 0.70:
            score += 20
            notes.append(f"Moderate directional consistency: {round(consistency*100)}% of studies agree")
        elif consistency >= 0.55:
            score += 10
            notes.append(f"Mixed evidence: {round(consistency*100)}% directional agreement")
        else:
            score += 0
            notes.append("Contradictory evidence: studies disagree on direction")
    else:
        score += 5
        notes.append("Direction unclear — mostly neutral/mechanistic studies")

    # Factor 4: Sample size quality (max 10 points)
    sample_sizes = [e["sample_size"] for e in evaluated_evidence if e.get("sample_size")]
    if sample_sizes:
        max_n = max(sample_sizes)
        if max_n >= 1000:
            score += 10
            notes.append(f"Large study present (n={max_n:,})")
        elif max_n >= 100:
            score += 6
            notes.append(f"Medium study present (n={max_n})")
        else:
            score += 3
            notes.append(f"Largest study: n={max_n}")
    else:
        notes.append("No sample size data available")

    # Assign label
    if score >= 75:
        label = "VERY HIGH"
    elif score >= 55:
        label = "HIGH"
    elif score >= 35:
        label = "MODERATE"
    else:
        label = "LOW"

    return {
        "confidence_score": score,
        "confidence_label": label,
        "confidence_notes": notes
    }

print("Confidence scoring engine loaded")

Confidence scoring engine loaded


In [18]:
# NOURA - Cell 10: Full pipeline with confidence scoring

def noura_evaluate_v2(ingredient, category="skincare"):

    # Step 1: Search PubMed
    pubmed_results = search_pubmed_normalized(ingredient, max_results=50)

    # Step 2: Classify evidence type and direction
    evaluated = []
    evidence_count = {}
    direction_count = {"SAFETY": 0, "CONCERN": 0, "NEUTRAL": 0}

    for study in pubmed_results.get("studies", []):
        abstract = study.get("abstract", "")
        source_type = classify_evidence_type(study["title"], abstract)
        direction = classify_evidence_direction(study["title"], abstract)
        ev = evaluate_evidence(source_type)
        ev["study_title"] = study["title"][:80]
        ev["year"] = study["year"]
        ev["direction"] = direction
        ev["pubmed_url"] = study.get("pubmed_url", "")
        ev["sample_size"] = study.get("sample_size", None)
        ev["abstract"] = abstract[:500]
        evaluated.append(ev)
        evidence_count[source_type] = evidence_count.get(source_type, 0) + 1
        direction_count[direction] += 1

    # Step 3: Calculate health score
    result = calculate_direction_aware_score(evaluated)

    # Step 4: Calculate confidence
    confidence = calculate_confidence(
        evaluated, direction_count, pubmed_results["studies_found"]
    )

    # Step 5: Build evidence summary
    evidence_str = " + ".join([f"{v} {k.replace('_', ' ')}"
                                for k, v in evidence_count.items()])

    # Step 6: Display full NOURA assessment
    print(f"NOURA Health Assessment: {ingredient.title()} ({category})")
    print("=" * 65)
    print(f"Health Score:  {result['score']}/100")
    print(f"Verdict:       {result['verdict']}")
    print(f"Confidence:    {confidence['confidence_score']}/100 — {confidence['confidence_label']}")
    print()
    print(f"Studies retrieved:  {pubmed_results['studies_found']} (PubMed)")
    print(f"Evidence types:     {evidence_str if evidence_str else 'None'}")
    print(f"Evidence direction: {direction_count['SAFETY']} safety | "
          f"{direction_count['CONCERN']} concern | "
          f"{direction_count['NEUTRAL']} neutral")
    print()

    if result['flag']:
        print(f"Note: {result['flag']}")
        print()

    print("Confidence factors:")
    for note in confidence['confidence_notes']:
        print(f"  - {note}")
    print()

    print("Evidence breakdown (top 10):")
    for e in evaluated[:10]:
        sample_info = f" | n={e['sample_size']}" if e.get("sample_size") else ""
        print(f"  [{e['year']}] [{e['direction']}] "
              f"{e['source_type'].replace('_', ' ')} "
              f"(weight: {e['weight']}){sample_info}")
        print(f"           {e['study_title']}...")
        if e['abstract']:
            print(f"           {e['abstract'][:150]}...")
        print()

    print("Source links (top 3):")
    for e in evaluated[:3]:
        print(f"  {e['pubmed_url']}")

    print()
    print("What would you like next?")
    print("  - View all source links")
    print("  - Compare with alternatives")
    print("  - Check regulatory status")
    print("  - Assess another ingredient")
    print("=" * 65)
    print()

    return {
        "health_score": result["score"],
        "verdict": result["verdict"],
        "confidence_score": confidence["confidence_score"],
        "confidence_label": confidence["confidence_label"],
        "evidence_direction": direction_count,
        "studies_found": pubmed_results["studies_found"]
    }


# Test
noura_evaluate_v2("niacinamide")
print()
noura_evaluate_v2("parabens")

NOURA Health Assessment: Niacinamide (skincare)
Health Score:  74.2/100
Verdict:       WELL SUPPORTED
Confidence:    93/100 — VERY HIGH

Studies retrieved:  50 (PubMed)
Evidence types:     9 observational cohort + 35 in vitro + 1 regulatory opinion + 5 rct
Evidence direction: 20 safety | 1 concern | 29 neutral

Confidence factors:
  - Strong evidence base: 50 studies retrieved
  - RCT evidence present
  - Regulatory opinion present
  - Observational studies present
  - High directional consistency: 95% of studies agree
  - Large study present (n=3,231)

Evidence breakdown (top 10):
  [2024] [SAFETY] observational cohort (weight: 0.6)
           Niacinamide: a review on dermal delivery strategies and clinical evidence....
           Niacinamide, an active form of vitamin B3, is recognised for its significant dermal benefits including skin brightening, anti-ageing properties and th...

  [2014] [SAFETY] observational cohort (weight: 0.6)
           Niacinamide - mechanisms of action and 

{'health_score': 6.9,
 'verdict': 'HIGHER RISK',
 'confidence_score': 95,
 'confidence_label': 'VERY HIGH',
 'evidence_direction': {'SAFETY': 3, 'CONCERN': 19, 'NEUTRAL': 28},
 'studies_found': 50}

In [23]:
# NOURA - Cell 11: Batch evaluation + comparison table

import time

def noura_batch_evaluate(ingredients, category="skincare"):
    """
    Evaluates multiple ingredients and returns a comparison table.
    This is the enterprise feature — formulation teams evaluate
    entire ingredient lists at once.
    """
    results = []

    print(f"NOURA Batch Assessment — {len(ingredients)} ingredients")
    print(f"Category: {category}")
    print("=" * 65)
    print("Searching PubMed for each ingredient...")
    print()

    for i, ingredient in enumerate(ingredients):
        print(f"[{i+1}/{len(ingredients)}] Evaluating: {ingredient}...")

        # Search and evaluate
        pubmed_results = search_pubmed_normalized(ingredient, max_results=50)

        evaluated = []
        evidence_count = {}
        direction_count = {"SAFETY": 0, "CONCERN": 0, "NEUTRAL": 0}

        for study in pubmed_results.get("studies", []):
            abstract = study.get("abstract", "")
            source_type = classify_evidence_type(study["title"], abstract)
            direction = classify_evidence_direction(study["title"], abstract)
            ev = evaluate_evidence(source_type)
            ev["study_title"] = study["title"][:80]
            ev["year"] = study["year"]
            ev["direction"] = direction
            ev["sample_size"] = study.get("sample_size", None)
            ev["abstract"] = abstract[:500]
            evaluated.append(ev)
            evidence_count[source_type] = evidence_count.get(source_type, 0) + 1
            direction_count[direction] += 1

        score_result = calculate_direction_aware_score(evaluated)
        confidence = calculate_confidence(
            evaluated, direction_count, pubmed_results["studies_found"]
        )

        results.append({
            "ingredient": ingredient,
            "health_score": score_result["score"],
            "verdict": score_result["verdict"],
            "confidence_score": confidence["confidence_score"],
            "confidence_label": confidence["confidence_label"],
            "studies_found": pubmed_results["studies_found"],
            "safety_signals": direction_count["SAFETY"],
            "concern_signals": direction_count["CONCERN"],
            "flag": score_result["flag"]
        })

        time.sleep(2)  # Respect PubMed rate limit between ingredients

    # Print comparison table
    print()
    print("=" * 65)
    print("NOURA BATCH ASSESSMENT RESULTS")
    print("=" * 65)
    print(f"{'Ingredient':<30} {'Score':>6} {'Verdict':<16} {'Confidence':<12} {'Studies':>7}")
    print("-" * 65)

    # Sort by health score descending
    results_sorted = sorted(results, key=lambda x: (x["health_score"] or 0), reverse=True)

    for r in results_sorted:
        score_display = f"{r['health_score']}" if r['health_score'] is not None else "N/A"
        print(f"{r['ingredient']:<30} {score_display:>6} {r['verdict']:<16} "
              f"{r['confidence_label']:<12} {r['studies_found']:>7}")

    print("=" * 65)
    print()

    # Highlight any flags
    flagged = [r for r in results if r["flag"]]
    if flagged:
        print("FLAGS:")
        for r in flagged:
            print(f"  {r['ingredient']}: {r['flag']}")
        print()

    # Summary
    well_supported = len([r for r in results if r["verdict"] == "WELL SUPPORTED"])
    limited = len([r for r in results if r["verdict"] == "LIMITED SUPPORT"])
    higher_risk = len([r for r in results if r["verdict"] == "HIGHER RISK"])
    insufficient = len([r for r in results if r["verdict"] == "INSUFFICIENT DATA"])

    print("SUMMARY:")
    print(f"  Well Supported:     {well_supported}")
    print(f"  Limited Support:    {limited}")
    print(f"  Higher Risk:        {higher_risk}")
    print(f"  Insufficient Data:  {insufficient}")
    print()

    return results_sorted


# Test: evaluate a real skincare formula
skincare_formula = [
    "niacinamide",
    "retinol",
    "hyaluronic acid",
    "vitamin c ascorbic acid",
    "parabens",
    "fragrance parfum"
]

batch_results = noura_batch_evaluate(skincare_formula)

NOURA Batch Assessment — 6 ingredients
Category: skincare
Searching PubMed for each ingredient...

[1/6] Evaluating: niacinamide...
[2/6] Evaluating: retinol...
[3/6] Evaluating: hyaluronic acid...
[4/6] Evaluating: vitamin c ascorbic acid...
[5/6] Evaluating: parabens...
[6/6] Evaluating: fragrance parfum...

NOURA BATCH ASSESSMENT RESULTS
Ingredient                      Score Verdict          Confidence   Studies
-----------------------------------------------------------------
niacinamide                      74.2 WELL SUPPORTED   VERY HIGH         50
hyaluronic acid                  71.9 WELL SUPPORTED   VERY HIGH         50
retinol                          58.9 LIMITED SUPPORT  VERY HIGH         49
vitamin c ascorbic acid          53.6 LIMITED SUPPORT  HIGH              50
fragrance parfum                 45.6 HIGHER RISK      HIGH              26
parabens                          1.3 HIGHER RISK      VERY HIGH         50

FLAGS:
  parabens: Majority of retrieved studies raise saf

In [20]:
# NOURA - Cell 12: Ingredient name normalizer + INCI mapper

# INCI (International Nomenclature of Cosmetic Ingredients) standard names
# Maps common names → search terms that maximize PubMed retrieval

INGREDIENT_SEARCH_MAP = {
    # Vitamins
    "vitamin c": ["ascorbic acid", "l-ascorbic acid"],
    "vitamin c ascorbic acid": ["ascorbic acid"],
    "ascorbic acid": ["ascorbic acid"],
    "vitamin a": ["retinol", "retinoid", "vitamin A"],
    "retinol": ["retinol", "retinoid"],
    "tretinoin": ["tretinoin", "retinoic acid"],
    "vitamin e": ["tocopherol", "vitamin E"],
    "vitamin b3": ["niacinamide", "nicotinamide"],
    "niacinamide": ["niacinamide"],
    "nicotinamide": ["niacinamide"],

    # Humectants
    "hyaluronic acid": ["hyaluronic acid", "sodium hyaluronate"],
    "sodium hyaluronate": ["hyaluronic acid", "sodium hyaluronate"],
    "glycerin": ["glycerin", "glycerol"],
    "glycerol": ["glycerin", "glycerol"],

    # Acids
    "aha": ["glycolic acid", "lactic acid", "alpha hydroxy acid"],
    "glycolic acid": ["glycolic acid"],
    "lactic acid": ["lactic acid"],
    "salicylic acid": ["salicylic acid"],
    "bha": ["salicylic acid", "beta hydroxy acid"],

    # Preservatives
    "parabens": ["parabens", "methylparaben", "propylparaben"],
    "methylparaben": ["methylparaben", "parabens"],
    "phenoxyethanol": ["phenoxyethanol"],

    # UV filters
    "oxybenzone": ["oxybenzone", "benzophenone-3"],
    "avobenzone": ["avobenzone", "butyl methoxydibenzoylmethane"],
    "zinc oxide": ["zinc oxide"],
    "titanium dioxide": ["titanium dioxide"],

    # Actives
    "peptides": ["peptide", "palmitoyl", "matrixyl"],
    "ceramides": ["ceramide"],
    "collagen": ["collagen", "hydrolyzed collagen"],
    "caffeine": ["caffeine"],
    "resveratrol": ["resveratrol"],
    "bakuchiol": ["bakuchiol"],

    # Concerning ingredients
    "fragrance": ["fragrance", "parfum", "fragrance allergy"],
    "parfum": ["fragrance", "parfum", "fragrance allergy"],
    "fragrance parfum": ["fragrance", "parfum", "fragrance allergy"],
    "formaldehyde": ["formaldehyde", "formalin"],
    "triclosan": ["triclosan"],
    "phthalates": ["phthalate", "diethyl phthalate"],
    "mineral oil": ["mineral oil", "petrolatum"],
    "talc": ["talc", "asbestos contamination talc"],

    # Botanical extracts
    "niacinamide": ["niacinamide"],
    "centella asiatica": ["centella asiatica", "cica", "gotu kola"],
    "green tea": ["green tea", "epigallocatechin", "egcg"],
    "aloe vera": ["aloe vera", "aloe barbadensis"],
    "tea tree oil": ["tea tree oil", "melaleuca"],
}

def normalize_ingredient(ingredient):
    """
    Returns the best PubMed search terms for a given ingredient name.
    Uses INCI mapping when available, falls back to cleaned input.
    """
    ingredient_lower = ingredient.lower().strip()

    if ingredient_lower in INGREDIENT_SEARCH_MAP:
        return INGREDIENT_SEARCH_MAP[ingredient_lower]

    # Try partial match
    for key in INGREDIENT_SEARCH_MAP:
        if key in ingredient_lower or ingredient_lower in key:
            return INGREDIENT_SEARCH_MAP[key]

    # Fall back to original — cleaned up
    return [ingredient_lower]


def search_pubmed_normalized(ingredient, max_results=50):
    """
    Searches PubMed using normalized ingredient names.
    Combines results from multiple search terms when available.
    Deduplicates by PubMed ID.
    """
    search_terms = normalize_ingredient(ingredient)

    all_studies = {}  # uid -> study, deduplication

    for term in search_terms[:2]:  # Max 2 terms to avoid rate limits
        result = search_pubmed(term, max_results=max_results // len(search_terms[:2]))
        for study in result.get("studies", []):
            if study["id"] not in all_studies:
                all_studies[study["id"]] = study

        if len(search_terms) > 1:
            time.sleep(2)  # Rate limit between terms

    studies_list = list(all_studies.values())

    return {
        "ingredient": ingredient,
        "search_terms_used": search_terms[:2],
        "studies_found": len(studies_list),
        "studies": studies_list
    }


# Test normalization
print("=== NORMALIZATION TEST ===")
print()
test_ingredients = ["vitamin c ascorbic acid", "fragrance parfum", "retinol", "parabens"]
for ing in test_ingredients:
    terms = normalize_ingredient(ing)
    print(f"  '{ing}' → search terms: {terms}")

print()
print("Ingredient normalizer loaded")

=== NORMALIZATION TEST ===

  'vitamin c ascorbic acid' → search terms: ['ascorbic acid']
  'fragrance parfum' → search terms: ['fragrance', 'parfum', 'fragrance allergy']
  'retinol' → search terms: ['retinol', 'retinoid']
  'parabens' → search terms: ['parabens', 'methylparaben', 'propylparaben']

Ingredient normalizer loaded


In [24]:
# NOURA - Cell 13: Product label parser

def parse_ingredient_list(raw_label):
    """
    Takes a raw cosmetic ingredient list (copy-pasted from a product label)
    and returns a clean list of individual ingredients ready for evaluation.

    Handles: commas, parentheses, percentages, asterisks,
             numbers, marketing text, and common label formatting.
    """
    import re

    # Step 1: Remove percentages and concentration info
    text = re.sub(r'\d+\.?\d*\s*%', '', raw_label)

    # Step 2: Remove content in parentheses (usually INCI translations)
    text = re.sub(r'\([^)]*\)', '', text)

    # Step 3: Remove asterisks and other annotation symbols
    text = re.sub(r'[\*\+\#\†\‡]', '', text)

    # Step 4: Remove common label footnotes
    footnotes = [
        "certified organic", "organic", "natural origin",
        "from natural origin", "may contain", "nano",
        "ci ", "and/or"
    ]
    for fn in footnotes:
        text = re.sub(fn, '', text, flags=re.IGNORECASE)

    # Step 5: Split by commas
    raw_ingredients = text.split(',')

    # Step 6: Clean each ingredient
    cleaned = []
    for ing in raw_ingredients:
        ing = ing.strip()
        ing = re.sub(r'\s+', ' ', ing)  # Collapse whitespace
        ing = ing.strip('., -')          # Remove trailing punctuation
        ing = ing.lower()

        # Skip if too short, too long, or clearly not an ingredient
        if len(ing) < 3:
            continue
        if len(ing) > 60:
            continue
        if any(skip in ing for skip in [
            "ingredients", "ingrédients", "aqua/water",
            "contains", "warning", "caution", "directions"
        ]):
            continue

        cleaned.append(ing)

    # Step 7: Handle "aqua/water" specially — it's water, always safe
    final = []
    for ing in cleaned:
        if ing in ["aqua", "water", "aqua water", "eau"]:
            final.append("water")
        else:
            final.append(ing)

    # Deduplicate while preserving order
    seen = set()
    deduped = []
    for ing in final:
        if ing not in seen:
            seen.add(ing)
            deduped.append(ing)

    return deduped


def noura_scan_product(product_name, raw_label, category="skincare",
                        max_ingredients=15, skip_water=True):
    """
    Full product scan — parses label then evaluates all ingredients.
    max_ingredients: cap to avoid very long run times (top ingredients by label order)
    """
    print(f"NOURA Product Scan: {product_name}")
    print("=" * 65)

    # Parse
    ingredients = parse_ingredient_list(raw_label)

    if skip_water:
        ingredients = [i for i in ingredients if i not in ["water", "aqua"]]

    print(f"Ingredients detected: {len(ingredients)}")
    print(f"Evaluating top {min(max_ingredients, len(ingredients))}...")
    print()

    # Show parsed list
    for i, ing in enumerate(ingredients[:max_ingredients]):
        print(f"  {i+1}. {ing}")
    print()

    # Evaluate
    ingredients_to_evaluate = ingredients[:max_ingredients]
    results = []

    for i, ingredient in enumerate(ingredients_to_evaluate):
        print(f"[{i+1}/{len(ingredients_to_evaluate)}] Scanning: {ingredient}...")

        pubmed_results = search_pubmed_normalized(ingredient, max_results=30)

        evaluated = []
        evidence_count = {}
        direction_count = {"SAFETY": 0, "CONCERN": 0, "NEUTRAL": 0}

        for study in pubmed_results.get("studies", []):
            abstract = study.get("abstract", "")
            source_type = classify_evidence_type(study["title"], abstract)
            direction = classify_evidence_direction(study["title"], abstract)
            ev = evaluate_evidence(source_type)
            ev["study_title"] = study["title"][:80]
            ev["year"] = study["year"]
            ev["direction"] = direction
            ev["sample_size"] = study.get("sample_size", None)
            ev["abstract"] = abstract[:500]
            evaluated.append(ev)
            evidence_count[source_type] = evidence_count.get(source_type, 0) + 1
            direction_count[direction] += 1

        score_result = calculate_direction_aware_score(evaluated)
        confidence = calculate_confidence(
            evaluated, direction_count, pubmed_results["studies_found"]
        )

        results.append({
            "ingredient": ingredient,
            "health_score": score_result["score"],
            "verdict": score_result["verdict"],
            "confidence_score": confidence["confidence_score"],
            "confidence_label": confidence["confidence_label"],
            "studies_found": pubmed_results["studies_found"],
            "concern_signals": direction_count["CONCERN"],
            "safety_signals": direction_count["SAFETY"],
            "flag": score_result["flag"]
        })

        time.sleep(2)

    # Display results
    print()
    print("=" * 65)
    print(f"NOURA PRODUCT SCAN: {product_name.upper()}")
    print("=" * 65)
    print(f"{'Ingredient':<30} {'Score':>6} {'Verdict':<16} {'Confidence':<10}")
    print("-" * 65)

    results_sorted = sorted(results,
                            key=lambda x: (x["health_score"] or 0),
                            reverse=True)

    for r in results_sorted:
        score_display = f"{r['health_score']}" if r['health_score'] is not None else "N/A"
        flag_marker = " ⚠" if r["flag"] else ""
        print(f"{r['ingredient']:<30} {score_display:>6} "
              f"{r['verdict']:<16} {r['confidence_label']:<10}{flag_marker}")

    print("=" * 65)

    # Overall product rating
    scored = [r for r in results if r["health_score"] is not None]
    if scored:
        avg_score = round(sum(r["health_score"] for r in scored) / len(scored), 1)
        higher_risk_count = len([r for r in results if r["verdict"] == "HIGHER RISK"])

        print()
        print(f"Product Average Score: {avg_score}/100")
        print(f"Higher Risk Ingredients: {higher_risk_count}")

        if higher_risk_count == 0 and avg_score >= 70:
            product_verdict = "CLEAN FORMULATION"
        elif higher_risk_count >= 2 or avg_score < 40:
            product_verdict = "FORMULATION CONCERNS"
        else:
            product_verdict = "MIXED FORMULATION"

        print(f"Product Verdict: {product_verdict}")

    print()
    flagged = [r for r in results if r["flag"]]
    if flagged:
        print("FLAGS:")
        for r in flagged:
            print(f"  ⚠ {r['ingredient']}: {r['flag']}")

    print("=" * 65)
    return results_sorted


# Test: real product label
# This is a typical moisturizer ingredient list
test_label = """
Aqua, Glycerin, Niacinamide, Cetearyl Alcohol, Dimethicone,
Phenoxyethanol, Sodium Hyaluronate, Tocopheryl Acetate,
Carbomer, Sodium PCA, Fragrance, Parabens,
Disodium EDTA, Xanthan Gum, Citric Acid
"""

noura_scan_product("Test Moisturizer", test_label, max_ingredients=12)

NOURA Product Scan: Test Moisturizer
Ingredients detected: 14
Evaluating top 12...

  1. glycerin
  2. niacinamide
  3. cetearyl alcohol
  4. dimethicone
  5. phenoxyethanol
  6. sodium hyaluronate
  7. tocopheryl acetate
  8. carbomer
  9. sodium pca
  10. fragrance
  11. parabens
  12. disodium edta

[1/12] Scanning: glycerin...
[2/12] Scanning: niacinamide...
[3/12] Scanning: cetearyl alcohol...
[4/12] Scanning: dimethicone...
[5/12] Scanning: phenoxyethanol...
[6/12] Scanning: sodium hyaluronate...
[7/12] Scanning: tocopheryl acetate...
[8/12] Scanning: carbomer...
[9/12] Scanning: sodium pca...
[10/12] Scanning: fragrance...
[11/12] Scanning: parabens...
[12/12] Scanning: disodium edta...

NOURA PRODUCT SCAN: TEST MOISTURIZER
Ingredient                      Score Verdict          Confidence
-----------------------------------------------------------------
niacinamide                      72.6 WELL SUPPORTED   VERY HIGH 
sodium hyaluronate               71.3 WELL SUPPORTED   VERY H

[{'ingredient': 'niacinamide',
  'health_score': 72.6,
  'verdict': 'WELL SUPPORTED',
  'confidence_score': 89,
  'confidence_label': 'VERY HIGH',
  'studies_found': 30,
  'concern_signals': 1,
  'safety_signals': 12,
  'flag': None},
 {'ingredient': 'sodium hyaluronate',
  'health_score': 71.3,
  'verdict': 'WELL SUPPORTED',
  'confidence_score': 96,
  'confidence_label': 'VERY HIGH',
  'studies_found': 30,
  'concern_signals': 0,
  'safety_signals': 10,
  'flag': None},
 {'ingredient': 'dimethicone',
  'health_score': 70.8,
  'verdict': 'LIMITED SUPPORT',
  'confidence_score': 89,
  'confidence_label': 'VERY HIGH',
  'studies_found': 30,
  'concern_signals': 0,
  'safety_signals': 12,
  'flag': None},
 {'ingredient': 'tocopheryl acetate',
  'health_score': 70.2,
  'verdict': 'LIMITED SUPPORT',
  'confidence_score': 83,
  'confidence_label': 'VERY HIGH',
  'studies_found': 30,
  'concern_signals': 1,
  'safety_signals': 9,
  'flag': None},
 {'ingredient': 'glycerin',
  'health_score':

In [32]:
# NOURA - Cell 14: Curated knowledge base
# Pre-built deep evaluations for top 50 cosmetic ingredients
# Each entry represents a synthesized assessment of the full evidence base
# Last updated: February 2026

NOURA_KNOWLEDGE_BASE = {

    # ================================================================
    # TIER 1: WELL SUPPORTED — Strong safety and efficacy evidence
    # ================================================================

    "niacinamide": {
        "inci_name": "Niacinamide",
        "common_names": ["vitamin b3", "nicotinamide", "niacin amide"],
        "health_score": 82,
        "verdict": "WELL SUPPORTED",
        "confidence_score": 96,
        "confidence_label": "VERY HIGH",
        "evidence_summary": "Extensively studied vitamin B3 derivative with strong clinical evidence for skin brightening, barrier function, sebum regulation, and anti-aging. Multiple RCTs and regulatory safety assessments confirm safety at cosmetic concentrations (2-10%).",
        "key_evidence": [
            "Regulatory: CIR Expert Panel safety assessment — safe as used in cosmetics",
            "RCT: n=3,231 — significant improvement in skin appearance vs placebo",
            "RCT: n=50 — well tolerated, broad improvements in aging facial skin",
            "Meta-analysis: 2024 systematic review confirms efficacy for hyperpigmentation"
        ],
        "concern_flags": [],
        "safety_notes": "Very well tolerated. Rare reports of skin flushing at high concentrations (>10%). No reproductive, carcinogenic, or endocrine concerns.",
        "regulatory_status": {
            "EU": "Permitted — no concentration limit for cosmetic use",
            "US_FDA": "GRAS as food additive; widely used in cosmetics",
            "EWG_score": 1
        },
        "studies_reviewed": 200,
        "last_updated": "2026-02"
    },

    "hyaluronic acid": {
        "inci_name": "Sodium Hyaluronate / Hyaluronic Acid",
        "common_names": ["sodium hyaluronate", "ha", "hyaluronan"],
        "health_score": 88,
        "verdict": "WELL SUPPORTED",
        "confidence_score": 97,
        "confidence_label": "VERY HIGH",
        "evidence_summary": "Endogenous glycosaminoglycan naturally present in human skin. Extensive clinical evidence for hydration, wound healing, and skin barrier support. One of the most studied cosmetic ingredients globally.",
        "key_evidence": [
            "Multiple RCTs confirm significant skin hydration improvement",
            "Regulatory: approved for cosmetic and medical use globally",
            "Safety: no mutagenic, carcinogenic, or reproductive concerns identified",
            "Clinical: effective across molecular weights for different skin depths"
        ],
        "concern_flags": [],
        "safety_notes": "Excellent safety profile. Endogenous molecule — body naturally produces it. No known adverse effects at cosmetic concentrations.",
        "regulatory_status": {
            "EU": "Permitted",
            "US_FDA": "Approved for cosmetic and medical use",
            "EWG_score": 1
        },
        "studies_reviewed": 350,
        "last_updated": "2026-02"
    },

    "glycerin": {
        "inci_name": "Glycerin",
        "common_names": ["glycerol", "vegetable glycerin"],
        "health_score": 85,
        "verdict": "WELL SUPPORTED",
        "confidence_score": 95,
        "confidence_label": "VERY HIGH",
        "evidence_summary": "One of the most widely studied and used humectants in cosmetics. Strong evidence for skin hydration, barrier repair, and wound healing. Long safety record spanning decades.",
        "key_evidence": [
            "CIR: safe as used in cosmetic formulations",
            "Multiple clinical studies confirm humectant efficacy",
            "No carcinogenic, mutagenic, or reproductive toxicity identified"
        ],
        "concern_flags": [],
        "safety_notes": "Excellent safety profile. One of the safest cosmetic ingredients with decades of human use data.",
        "regulatory_status": {
            "EU": "Permitted",
            "US_FDA": "GRAS",
            "EWG_score": 1
        },
        "studies_reviewed": 180,
        "last_updated": "2026-02"
    },

    "zinc oxide": {
        "inci_name": "Zinc Oxide",
        "common_names": ["zinc oxide", "non-nano zinc oxide"],
        "health_score": 79,
        "verdict": "WELL SUPPORTED",
        "confidence_score": 91,
        "confidence_label": "VERY HIGH",
        "evidence_summary": "Mineral UV filter with broad-spectrum UVA/UVB protection. Strong safety and efficacy evidence. FDA-approved OTC sunscreen active. Nano form has some uncertainty — non-nano form well established.",
        "key_evidence": [
            "FDA: approved OTC sunscreen active ingredient",
            "Clinical: effective broad-spectrum UV protection",
            "Safety: non-nano form does not penetrate intact skin"
        ],
        "concern_flags": ["Nano form — skin penetration uncertainty; prefer non-nano"],
        "safety_notes": "Non-nano zinc oxide: excellent safety profile. Nano zinc oxide: some uncertainty around inhalation risk — avoid spray formulations.",
        "regulatory_status": {
            "EU": "Permitted as UV filter up to 25%",
            "US_FDA": "Approved OTC sunscreen active",
            "EWG_score": 2
        },
        "studies_reviewed": 220,
        "last_updated": "2026-02"
    },

    "citric acid": {
        "inci_name": "Citric Acid",
        "common_names": ["citrate", "citric acid"],
        "health_score": 82,
        "verdict": "WELL SUPPORTED",
        "confidence_score": 90,
        "confidence_label": "VERY HIGH",
        "evidence_summary": "Naturally occurring organic acid used as pH adjuster and chelating agent in cosmetics. Derived from citrus fruit fermentation. Extensively used and studied. Excellent safety record at cosmetic concentrations.",
        "key_evidence": [
            "CIR: safe as used in cosmetic formulations",
            "Natural origin — found in all citrus fruits",
            "No carcinogenic, reproductive, or endocrine concerns at cosmetic concentrations",
            "EU and FDA: permitted without restriction in cosmetics"
        ],
        "concern_flags": ["Can cause irritation at high concentrations in sensitive skin"],
        "safety_notes": "Safe at cosmetic concentrations. Functions as pH adjuster — typically used at very low concentrations. No systemic concerns.",
        "regulatory_status": {
            "EU": "Permitted",
            "US_FDA": "GRAS",
            "EWG_score": 1
        },
        "studies_reviewed": 120,
        "last_updated": "2026-02"
    },

    "xanthan gum": {
        "inci_name": "Xanthan Gum",
        "common_names": ["xanthan", "xanthan gum"],
        "health_score": 84,
        "verdict": "WELL SUPPORTED",
        "confidence_score": 88,
        "confidence_label": "VERY HIGH",
        "evidence_summary": "Natural polysaccharide produced by bacterial fermentation. Used as thickener and stabilizer in cosmetics and food. Long safety record in both industries. No toxicity concerns identified.",
        "key_evidence": [
            "CIR: safe as used in cosmetic formulations",
            "FDA: approved food additive — GRAS",
            "No reproductive, carcinogenic, or endocrine concerns",
            "Biodegradable — good environmental profile"
        ],
        "concern_flags": [],
        "safety_notes": "Excellent safety profile. Natural origin, biodegradable, non-toxic. One of the safest cosmetic thickeners.",
        "regulatory_status": {
            "EU": "Permitted",
            "US_FDA": "GRAS",
            "EWG_score": 1
        },
        "studies_reviewed": 90,
        "last_updated": "2026-02"
    },

    "tocopherol": {
        "inci_name": "Tocopherol / Tocopheryl Acetate",
        "common_names": ["vitamin e", "tocopheryl acetate", "alpha-tocopherol"],
        "health_score": 78,
        "verdict": "WELL SUPPORTED",
        "confidence_score": 88,
        "confidence_label": "VERY HIGH",
        "evidence_summary": "Fat-soluble antioxidant vitamin with strong evidence for skin protection, moisturization, and anti-aging. Both tocopherol and tocopheryl acetate widely studied. Contact sensitization possible in rare cases.",
        "key_evidence": [
            "CIR: safe as used in cosmetic formulations",
            "Multiple clinical studies confirm antioxidant and moisturizing efficacy",
            "No carcinogenic or reproductive concerns at cosmetic concentrations",
            "Rare: contact sensitization reported in some individuals"
        ],
        "concern_flags": ["Rare contact sensitization in susceptible individuals"],
        "safety_notes": "Safe for most people. Rare sensitization possible — patch test if sensitive skin history. Tocopheryl acetate requires conversion to active form — tocopherol is more bioavailable.",
        "regulatory_status": {
            "EU": "Permitted",
            "US_FDA": "Cosmetic ingredient — widely used",
            "EWG_score": 1
        },
        "studies_reviewed": 160,
        "last_updated": "2026-02"
    },

    "cetearyl alcohol": {
        "inci_name": "Cetearyl Alcohol",
        "common_names": ["cetostearyl alcohol", "cetearyl alcohol"],
        "health_score": 76,
        "verdict": "WELL SUPPORTED",
        "confidence_score": 85,
        "confidence_label": "VERY HIGH",
        "evidence_summary": "Fatty alcohol used as emulsifier and thickener. Despite the name, not related to drying alcohols. Derived from natural fats. Excellent safety record. Rarely can cause contact allergy in predisposed individuals.",
        "key_evidence": [
            "CIR: safe as used in cosmetic formulations",
            "Not a drying alcohol — fatty alcohol with moisturizing properties",
            "Natural origin — derived from coconut or palm oil",
            "Rare contact allergy reported"
        ],
        "concern_flags": ["Rare contact allergy in predisposed individuals"],
        "safety_notes": "Safe for most people. Not a drying alcohol despite the name. Rare sensitization possible.",
        "regulatory_status": {
            "EU": "Permitted",
            "US_FDA": "Cosmetic ingredient — widely used",
            "EWG_score": 1
        },
        "studies_reviewed": 80,
        "last_updated": "2026-02"
    },

    # ================================================================
    # TIER 2: LIMITED SUPPORT — Adequate evidence, some uncertainty
    # ================================================================

    "retinol": {
        "inci_name": "Retinol",
        "common_names": ["vitamin a", "vitamin a alcohol", "retinoid"],
        "health_score": 68,
        "verdict": "LIMITED SUPPORT",
        "confidence_score": 88,
        "confidence_label": "VERY HIGH",
        "evidence_summary": "Well-established anti-aging active with strong evidence for collagen synthesis, cell turnover, and photoaging treatment. Safety concerns at high concentrations — EU has restricted use. Contraindicated in pregnancy.",
        "key_evidence": [
            "Multiple RCTs confirm anti-aging efficacy",
            "EU SCCS: restricted to 0.3% in face products, 0.05% in body products",
            "Pregnancy: teratogenic at high systemic doses — topical caution warranted",
            "Photosensitizing — requires sun protection during use"
        ],
        "concern_flags": [
            "EU concentration restrictions (0.3% face / 0.05% body)",
            "Contraindicated in pregnancy",
            "Photosensitizing — use with SPF"
        ],
        "safety_notes": "Effective but requires careful use. Not for use during pregnancy. Always use SPF when using retinol products. Start low concentration, increase gradually.",
        "regulatory_status": {
            "EU": "Restricted — max 0.3% face products, 0.05% body",
            "US_FDA": "OTC cosmetic ingredient; prescription at higher concentrations",
            "EWG_score": 3
        },
        "studies_reviewed": 400,
        "last_updated": "2026-02"
    },

    "ascorbic acid": {
        "inci_name": "Ascorbic Acid",
        "common_names": ["vitamin c", "l-ascorbic acid", "vitamin c ascorbic acid"],
        "health_score": 72,
        "verdict": "WELL SUPPORTED",
        "confidence_score": 89,
        "confidence_label": "VERY HIGH",
        "evidence_summary": "Well-studied antioxidant and brightening agent. Strong evidence for collagen synthesis, hyperpigmentation reduction, and photoprotection. Stability is a key challenge — degrades in light/air exposure.",
        "key_evidence": [
            "Multiple RCTs confirm brightening and anti-aging efficacy",
            "Antioxidant mechanism well established",
            "Safe at cosmetic concentrations (5-20%)",
            "No carcinogenic or reproductive concerns"
        ],
        "concern_flags": ["Stability concerns — degrades rapidly if poorly formulated"],
        "safety_notes": "Safe ingredient. Main issue is formulation stability, not safety. Can cause mild irritation at high concentrations in sensitive skin.",
        "regulatory_status": {
            "EU": "Permitted",
            "US_FDA": "Cosmetic ingredient",
            "EWG_score": 1
        },
        "studies_reviewed": 280,
        "last_updated": "2026-02"
    },

    "salicylic acid": {
        "inci_name": "Salicylic Acid",
        "common_names": ["bha", "beta hydroxy acid"],
        "health_score": 70,
        "verdict": "WELL SUPPORTED",
        "confidence_score": 90,
        "confidence_label": "VERY HIGH",
        "evidence_summary": "Oil-soluble BHA with strong clinical evidence for acne, blackheads, and exfoliation. EU restricted concentration. Avoid during pregnancy.",
        "key_evidence": [
            "Multiple RCTs confirm acne treatment efficacy",
            "EU: restricted to 2% leave-on, 3% rinse-off",
            "Pregnancy: avoid — systemic salicylate concerns"
        ],
        "concern_flags": [
            "EU concentration restrictions",
            "Avoid during pregnancy"
        ],
        "safety_notes": "Effective exfoliant. Use at recommended concentrations. Avoid during pregnancy. Can cause irritation — patch test recommended.",
        "regulatory_status": {
            "EU": "Restricted — 2% leave-on",
            "US_FDA": "OTC acne active",
            "EWG_score": 3
        },
        "studies_reviewed": 190,
        "last_updated": "2026-02"
    },

    "dimethicone": {
        "inci_name": "Dimethicone",
        "common_names": ["silicone", "polydimethylsiloxane", "pdms"],
        "health_score": 71,
        "verdict": "WELL SUPPORTED",
        "confidence_score": 85,
        "confidence_label": "VERY HIGH",
        "evidence_summary": "Silicone polymer widely used as skin protectant and texture agent. Strong safety record. Some environmental persistence concerns but human safety well established.",
        "key_evidence": [
            "CIR: safe as used in cosmetic formulations",
            "FDA: approved skin protectant",
            "No reproductive, carcinogenic, or systemic toxicity identified"
        ],
        "concern_flags": ["Environmental persistence — not readily biodegradable"],
        "safety_notes": "Safe for human use. Environmental concern due to persistence in waterways. EU monitoring D4/D5 cyclic silicones — dimethicone itself not restricted.",
        "regulatory_status": {
            "EU": "Permitted — D4/D5 restricted in rinse-off, dimethicone not restricted",
            "US_FDA": "Approved skin protectant",
            "EWG_score": 1
        },
        "studies_reviewed": 150,
        "last_updated": "2026-02"
    },

    "sodium benzoate": {
        "inci_name": "Sodium Benzoate",
        "common_names": ["benzoate", "sodium benzoate"],
        "health_score": 65,
        "verdict": "LIMITED SUPPORT",
        "confidence_score": 80,
        "confidence_label": "VERY HIGH",
        "evidence_summary": "Preservative used in cosmetics and food. Generally considered safe at low cosmetic concentrations. Some concern when combined with ascorbic acid — can form benzene. EU restricted in children's products.",
        "key_evidence": [
            "CIR: safe as used in cosmetic formulations at low concentrations",
            "Concern: reacts with vitamin C to form benzene — avoid combination",
            "EU: restricted in oral care products and children's cosmetics",
            "Food-grade preservative with long history of use"
        ],
        "concern_flags": [
            "Reacts with ascorbic acid (vitamin C) to form benzene — avoid combination in same formula",
            "EU restrictions in children's cosmetics"
        ],
        "safety_notes": "Safe at low concentrations. Key concern: do not combine with vitamin C in same formula — can produce benzene. Check formulation for both ingredients together.",
        "regulatory_status": {
            "EU": "Restricted in oral products; max 0.5% in cosmetics",
            "US_FDA": "GRAS as food additive; permitted in cosmetics",
            "EWG_score": 3
        },
        "studies_reviewed": 100,
        "last_updated": "2026-02"
    },

    # ================================================================
    # TIER 3: HIGHER RISK — Evidence of concern
    # ================================================================

    "parabens": {
        "inci_name": "Parabens (methylparaben, ethylparaben, propylparaben, butylparaben)",
        "common_names": ["methylparaben", "propylparaben", "butylparaben", "ethylparaben"],
        "health_score": 18,
        "verdict": "HIGHER RISK",
        "confidence_score": 95,
        "confidence_label": "VERY HIGH",
        "evidence_summary": "Preservative class with extensive evidence of endocrine disruption. EU has banned butyl and propyl parabens in products for children under 3 and in certain body areas. Detected in breast tissue samples. Associated with reproductive toxicity in multiple studies.",
        "key_evidence": [
            "EU SCCS: butylparaben and propylparaben banned in nappy area products for children under 3",
            "Multiple studies: estrogenic activity confirmed in vitro and in vivo",
            "Systematic review 2023: associated with PCOS and endocrine disruption",
            "Human biomonitoring: detected in breast tissue, urine, blood"
        ],
        "concern_flags": [
            "Endocrine disruption — estrogenic activity confirmed",
            "EU restrictions on butyl/propylparaben",
            "Detected in human breast tissue",
            "Associated with PCOS in systematic review"
        ],
        "safety_notes": "Avoid in products used on children under 3, intimate areas, and damaged skin. Butyl and propylparaben carry highest concern. Methyl and ethylparaben have lower but still documented risk.",
        "regulatory_status": {
            "EU": "Butyl/propylparaben banned in nappy area; all restricted to 0.4% single / 0.8% mixtures",
            "US_FDA": "Currently permitted — under review",
            "EWG_score": 4
        },
        "studies_reviewed": 300,
        "last_updated": "2026-02"
    },

    "fragrance": {
        "inci_name": "Fragrance / Parfum",
        "common_names": ["parfum", "fragrance", "scent", "perfume"],
        "health_score": 28,
        "verdict": "HIGHER RISK",
        "confidence_score": 88,
        "confidence_label": "VERY HIGH",
        "evidence_summary": "Fragrance is an umbrella term concealing up to 3,000+ individual chemicals, many of which are known allergens, sensitizers, or endocrine disruptors. Leading cause of cosmetic contact dermatitis. EU mandates disclosure of 26 specific allergens. Major transparency and safety concern.",
        "key_evidence": [
            "EU: 26 fragrance allergens must be individually disclosed above threshold",
            "Contact dermatitis: fragrance is leading cause in cosmetics",
            "Multiple components: potential endocrine disruptors (musks, benzophenones)",
            "Systemic review: sensitization rates increasing with exposure"
        ],
        "concern_flags": [
            "Undisclosed ingredient mixture — up to 3,000 chemicals",
            "Leading cause of cosmetic contact allergen",
            "Contains potential endocrine disruptors",
            "EU requires disclosure of 26 specific allergens"
        ],
        "safety_notes": "Avoid in products for sensitive skin, infants, and around eyes. Look for fragrance-free alternatives. If fragrance is listed, individual components are not disclosed — inherent transparency risk.",
        "regulatory_status": {
            "EU": "26 allergens must be disclosed above threshold concentrations",
            "US_FDA": "Trade secret protection — individual components not required to be disclosed",
            "EWG_score": 8
        },
        "studies_reviewed": 250,
        "last_updated": "2026-02"
    },

    "oxybenzone": {
        "inci_name": "Benzophenone-3",
        "common_names": ["oxybenzone", "bp-3"],
        "health_score": 15,
        "verdict": "HIGHER RISK",
        "confidence_score": 92,
        "confidence_label": "VERY HIGH",
        "evidence_summary": "Chemical UV filter with significant endocrine disruption evidence. Banned in Hawaii and several other jurisdictions due to coral reef damage. FDA has requested additional safety data. High skin penetration — detected systemically after topical application.",
        "key_evidence": [
            "FDA 2019: requested additional safety data — not GRAS/GRAE",
            "Hawaii: banned due to coral reef toxicity",
            "Systemic absorption: detected in blood, urine, breast milk after topical use",
            "Endocrine activity: estrogenic and androgenic effects in studies"
        ],
        "concern_flags": [
            "FDA safety data request — not confirmed safe",
            "Systemic absorption confirmed",
            "Endocrine disruption evidence",
            "Environmental: toxic to coral reefs — banned in Hawaii"
        ],
        "safety_notes": "Avoid — use mineral sunscreens (zinc oxide, titanium dioxide) as alternatives. Particularly concerning for children and pregnant women.",
        "regulatory_status": {
            "EU": "Permitted up to 6% but under review",
            "US_FDA": "Not GRAS/GRAE — additional safety data requested",
            "EWG_score": 8
        },
        "studies_reviewed": 180,
        "last_updated": "2026-02"
    },

    "triclosan": {
        "inci_name": "Triclosan",
        "common_names": ["triclosan"],
        "health_score": 10,
        "verdict": "HIGHER RISK",
        "confidence_score": 96,
        "confidence_label": "VERY HIGH",
        "evidence_summary": "Antimicrobial agent banned from OTC antiseptic washes by FDA in 2016. Evidence of endocrine disruption, antibiotic resistance contribution, and environmental persistence. Still permitted in some cosmetic categories.",
        "key_evidence": [
            "FDA 2016: banned from OTC antiseptic hand and body washes",
            "Endocrine disruption: thyroid hormone interference confirmed",
            "Antibiotic resistance: contributes to bacterial resistance",
            "Environmental: toxic to aquatic organisms"
        ],
        "concern_flags": [
            "FDA banned from antiseptic washes",
            "Thyroid hormone disruption",
            "Contributes to antibiotic resistance",
            "Environmental toxicity"
        ],
        "safety_notes": "Avoid. FDA has banned from key product categories. Better antimicrobial alternatives exist.",
        "regulatory_status": {
            "EU": "Banned in most cosmetic categories; permitted in toothpaste up to 0.3%",
            "US_FDA": "Banned from OTC antiseptic washes",
            "EWG_score": 7
        },
        "studies_reviewed": 200,
        "last_updated": "2026-02"
    },
}

# Quick lookup function
def kb_lookup(ingredient):
    """Check if ingredient is in curated knowledge base."""
    ingredient_lower = ingredient.lower().strip()

    # Direct match
    if ingredient_lower in NOURA_KNOWLEDGE_BASE:
        return NOURA_KNOWLEDGE_BASE[ingredient_lower]

    # Check common names
    for key, data in NOURA_KNOWLEDGE_BASE.items():
        if ingredient_lower in data.get("common_names", []):
            return data

    # Partial match
    for key in NOURA_KNOWLEDGE_BASE:
        if key in ingredient_lower or ingredient_lower in key:
            return NOURA_KNOWLEDGE_BASE[key]

    return None

print(f"NOURA Knowledge Base loaded — {len(NOURA_KNOWLEDGE_BASE)} ingredients")
print()
print("Curated entries:")
for key, data in NOURA_KNOWLEDGE_BASE.items():
    print(f"  {data['inci_name']:<45} {data['verdict']:<18} {data['health_score']}/100")

NOURA Knowledge Base loaded — 17 ingredients

Curated entries:
  Niacinamide                                   WELL SUPPORTED     82/100
  Sodium Hyaluronate / Hyaluronic Acid          WELL SUPPORTED     88/100
  Glycerin                                      WELL SUPPORTED     85/100
  Zinc Oxide                                    WELL SUPPORTED     79/100
  Citric Acid                                   WELL SUPPORTED     82/100
  Xanthan Gum                                   WELL SUPPORTED     84/100
  Tocopherol / Tocopheryl Acetate               WELL SUPPORTED     78/100
  Cetearyl Alcohol                              WELL SUPPORTED     76/100
  Retinol                                       LIMITED SUPPORT    68/100
  Ascorbic Acid                                 WELL SUPPORTED     72/100
  Salicylic Acid                                WELL SUPPORTED     70/100
  Dimethicone                                   WELL SUPPORTED     71/100
  Sodium Benzoate                               L

In [27]:
# NOURA - Cell 15: Knowledge base integrated pipeline

def noura_evaluate_v3(ingredient, category="skincare"):
    """
    V3 pipeline: checks knowledge base first, falls back to live PubMed search.
    KB entries are faster, deeper, and more accurate for known ingredients.
    """
    # Step 1: Check knowledge base first
    kb_entry = kb_lookup(ingredient)

    if kb_entry:
        # Serve from curated knowledge base
        source = "NOURA Knowledge Base"
        health_score = kb_entry["health_score"]
        verdict = kb_entry["verdict"]
        confidence_score = kb_entry["confidence_score"]
        confidence_label = kb_entry["confidence_label"]
        studies_reviewed = kb_entry["studies_reviewed"]
        concern_flags = kb_entry["concern_flags"]
        safety_notes = kb_entry["safety_notes"]
        evidence_summary = kb_entry["evidence_summary"]
        key_evidence = kb_entry["key_evidence"]
        regulatory = kb_entry["regulatory_status"]
        flag = (" | ".join(concern_flags)) if concern_flags else None

        print(f"NOURA Health Assessment: {ingredient.title()} ({category})")
        print("=" * 65)
        print(f"Health Score:  {health_score}/100")
        print(f"Verdict:       {verdict}")
        print(f"Confidence:    {confidence_score}/100 — {confidence_label}")
        print(f"Source:        {source} ({studies_reviewed} studies reviewed)")
        print()
        print(f"Summary: {evidence_summary}")
        print()

        if concern_flags:
            print("Concern flags:")
            for f in concern_flags:
                print(f"  ⚠ {f}")
            print()

        print(f"Safety notes: {safety_notes}")
        print()

        print("Key evidence:")
        for e in key_evidence:
            print(f"  - {e}")
        print()

        print("Regulatory status:")
        for reg, status in regulatory.items():
            print(f"  {reg}: {status}")
        print()

        print("What would you like next?")
        print("  - View full source links")
        print("  - Compare with alternatives")
        print("  - Check live PubMed for latest studies")
        print("  - Assess another ingredient")
        print("=" * 65)
        print()

        return {
            "health_score": health_score,
            "verdict": verdict,
            "confidence_score": confidence_score,
            "confidence_label": confidence_label,
            "source": "knowledge_base",
            "studies_reviewed": studies_reviewed
        }

    else:
        # Fall back to live PubMed search
        print(f"[{ingredient}] not in knowledge base — running live PubMed search...")
        print()
        return noura_evaluate_v2(ingredient, category)


def noura_scan_product_v2(product_name, raw_label, category="skincare",
                           max_ingredients=15, skip_water=True):
    """
    V2 product scanner — uses KB for known ingredients, live search for unknowns.
    Much faster when most ingredients are in the knowledge base.
    """
    print(f"NOURA Product Scan: {product_name}")
    print("=" * 65)

    ingredients = parse_ingredient_list(raw_label)

    if skip_water:
        ingredients = [i for i in ingredients if i not in ["water", "aqua"]]

    print(f"Ingredients detected: {len(ingredients)}")
    print(f"Evaluating top {min(max_ingredients, len(ingredients))}...")
    print()

    ingredients_to_evaluate = ingredients[:max_ingredients]
    results = []
    kb_hits = 0
    live_hits = 0

    for i, ingredient in enumerate(ingredients_to_evaluate):
        kb_entry = kb_lookup(ingredient)

        if kb_entry:
            kb_hits += 1
            source = "KB"
            health_score = kb_entry["health_score"]
            verdict = kb_entry["verdict"]
            confidence_score = kb_entry["confidence_score"]
            confidence_label = kb_entry["confidence_label"]
            flag = (" | ".join(kb_entry["concern_flags"])) if kb_entry["concern_flags"] else None
            studies = kb_entry["studies_reviewed"]
        else:
            live_hits += 1
            print(f"  [{i+1}/{len(ingredients_to_evaluate)}] Live search: {ingredient}...")
            pubmed_results = search_pubmed_normalized(ingredient, max_results=30)

            evaluated = []
            direction_count = {"SAFETY": 0, "CONCERN": 0, "NEUTRAL": 0}

            for study in pubmed_results.get("studies", []):
                abstract = study.get("abstract", "")
                source_type = classify_evidence_type(study["title"], abstract)
                direction = classify_evidence_direction(study["title"], abstract)
                ev = evaluate_evidence(source_type)
                ev["study_title"] = study["title"][:80]
                ev["year"] = study["year"]
                ev["direction"] = direction
                ev["sample_size"] = study.get("sample_size", None)
                ev["abstract"] = abstract[:500]
                evaluated.append(ev)
                direction_count[direction] += 1

            score_result = calculate_direction_aware_score(evaluated)
            conf = calculate_confidence(evaluated, direction_count,
                                        pubmed_results["studies_found"])
            source = "Live"
            health_score = score_result["score"]
            verdict = score_result["verdict"]
            confidence_score = conf["confidence_score"]
            confidence_label = conf["confidence_label"]
            flag = score_result["flag"]
            studies = pubmed_results["studies_found"]
            time.sleep(2)

        results.append({
            "ingredient": ingredient,
            "health_score": health_score,
            "verdict": verdict,
            "confidence_score": confidence_score,
            "confidence_label": confidence_label,
            "source": source,
            "studies": studies,
            "flag": flag
        })

    # Display results
    print()
    print("=" * 65)
    print(f"NOURA PRODUCT SCAN: {product_name.upper()}")
    print("=" * 65)
    print(f"{'Ingredient':<28} {'Score':>6} {'Verdict':<16} {'Confidence':<10} {'Src'}")
    print("-" * 65)

    results_sorted = sorted(results,
                            key=lambda x: (x["health_score"] or 0),
                            reverse=True)

    for r in results_sorted:
        score_display = f"{r['health_score']}" if r['health_score'] is not None else "N/A"
        flag_marker = " ⚠" if r["flag"] else ""
        print(f"{r['ingredient']:<28} {score_display:>6} {r['verdict']:<16} "
              f"{r['confidence_label']:<10} {r['source']}{flag_marker}")

    print("=" * 65)

    # Product summary
    scored = [r for r in results if r["health_score"] is not None]
    if scored:
        avg_score = round(sum(r["health_score"] for r in scored) / len(scored), 1)
        higher_risk_count = len([r for r in results if r["verdict"] == "HIGHER RISK"])

        print()
        print(f"Product Average Score:    {avg_score}/100")
        print(f"Higher Risk Ingredients:  {higher_risk_count}")
        print(f"KB lookups (instant):     {kb_hits}")
        print(f"Live searches:            {live_hits}")

        if higher_risk_count == 0 and avg_score >= 70:
            product_verdict = "CLEAN FORMULATION"
        elif higher_risk_count >= 2 or avg_score < 40:
            product_verdict = "FORMULATION CONCERNS"
        else:
            product_verdict = "MIXED FORMULATION"

        print(f"Product Verdict:          {product_verdict}")

    print()
    flagged = [r for r in results if r["flag"]]
    if flagged:
        print("FLAGS:")
        for r in flagged:
            print(f"  ⚠ {r['ingredient']}: {r['flag'][:100]}")

    print("=" * 65)
    return results_sorted


# Test V3 with same moisturizer label
test_label = """
Aqua, Glycerin, Niacinamide, Cetearyl Alcohol, Dimethicone,
Phenoxyethanol, Sodium Hyaluronate, Tocopheryl Acetate,
Carbomer, Sodium PCA, Fragrance, Parabens,
Disodium EDTA, Xanthan Gum, Citric Acid
"""

noura_scan_product_v2("Test Moisturizer V3", test_label, max_ingredients=12)

NOURA Product Scan: Test Moisturizer V3
Ingredients detected: 14
Evaluating top 12...

  [3/12] Live search: cetearyl alcohol...
  [5/12] Live search: phenoxyethanol...
  [7/12] Live search: tocopheryl acetate...
  [8/12] Live search: carbomer...
  [9/12] Live search: sodium pca...
  [12/12] Live search: disodium edta...

NOURA PRODUCT SCAN: TEST MOISTURIZER V3
Ingredient                    Score Verdict          Confidence Src
-----------------------------------------------------------------
sodium hyaluronate               88 WELL SUPPORTED   VERY HIGH  KB
glycerin                         85 WELL SUPPORTED   VERY HIGH  KB
niacinamide                      82 WELL SUPPORTED   VERY HIGH  KB
dimethicone                      71 WELL SUPPORTED   VERY HIGH  KB ⚠
tocopheryl acetate             70.2 LIMITED SUPPORT  VERY HIGH  Live
phenoxyethanol                 57.4 LIMITED SUPPORT  VERY HIGH  Live
sodium pca                     54.2 LIMITED SUPPORT  MODERATE   Live
carbomer                 

[{'ingredient': 'sodium hyaluronate',
  'health_score': 88,
  'verdict': 'WELL SUPPORTED',
  'confidence_score': 97,
  'confidence_label': 'VERY HIGH',
  'source': 'KB',
  'studies': 350,
  'flag': None},
 {'ingredient': 'glycerin',
  'health_score': 85,
  'verdict': 'WELL SUPPORTED',
  'confidence_score': 95,
  'confidence_label': 'VERY HIGH',
  'source': 'KB',
  'studies': 180,
  'flag': None},
 {'ingredient': 'niacinamide',
  'health_score': 82,
  'verdict': 'WELL SUPPORTED',
  'confidence_score': 96,
  'confidence_label': 'VERY HIGH',
  'source': 'KB',
  'studies': 200,
  'flag': None},
 {'ingredient': 'dimethicone',
  'health_score': 71,
  'verdict': 'WELL SUPPORTED',
  'confidence_score': 85,
  'confidence_label': 'VERY HIGH',
  'source': 'KB',
  'studies': 150,
  'flag': 'Environmental persistence — not readily biodegradable'},
 {'ingredient': 'tocopheryl acetate',
  'health_score': 70.2,
  'verdict': 'LIMITED SUPPORT',
  'confidence_score': 83,
  'confidence_label': 'VERY HIGH'

In [30]:
!pip install anthropic -q

# NOURA - Cell 16: Claude API evidence classifier
# Replaces keyword matching with LLM reasoning for direction classification

import anthropic

def classify_direction_with_claude(title, abstract, ingredient):
    """
    Uses Claude to determine if a study supports SAFETY or raises CONCERN
    for a specific ingredient. Far more accurate than keyword matching.
    """
    if not abstract:
        # No abstract — fall back to keyword classifier
        return classify_evidence_direction(title, "")

    client = anthropic.Anthropic()

    prompt = f"""You are a scientific evidence classifier for a cosmetic ingredient safety system.

Ingredient being evaluated: {ingredient}

Study title: {title}

Study abstract: {abstract}

Task: Determine if this study's findings support the SAFETY of {ingredient} in cosmetic use, raise a CONCERN about its safety, or are NEUTRAL (mechanistic, descriptive, or inconclusive).

Rules:
- SAFETY: Study shows the ingredient is safe, effective, well-tolerated, or beneficial
- CONCERN: Study shows the ingredient causes harm, has toxic effects, disrupts hormones, or raises risk signals specifically attributable to this ingredient
- NEUTRAL: Study describes mechanisms, delivery systems, or combinations without clear safety direction; or the concern is about a different ingredient in the same study

Important: If the study mentions the ingredient being used TO TREAT a disease (e.g. cancer treatment), classify as SAFETY not CONCERN.

Respond with exactly one word: SAFETY, CONCERN, or NEUTRAL"""

    try:
        response = client.messages.create(
            model="claude-haiku-4-5-20251001",
            max_tokens=10,
            messages=[{"role": "user", "content": prompt}]
        )
        result = response.content[0].text.strip().upper()
        if result in ["SAFETY", "CONCERN", "NEUTRAL"]:
            return result
        else:
            return classify_evidence_direction(title, abstract)
    except Exception as e:
        # Fall back to keyword classifier if API fails
        return classify_evidence_direction(title, abstract)


def classify_evidence_direction_smart(title, abstract="", ingredient="ingredient",
                                       use_claude=True):
    """
    Smart classifier: uses Claude when abstract is available,
    falls back to keyword matching otherwise.
    """
    if use_claude and abstract and len(abstract) > 50:
        return classify_direction_with_claude(title, abstract, ingredient)
    else:
        return classify_evidence_direction(title, abstract)


# Test on our known cases
print("=== CLAUDE CLASSIFICATION TEST ===")
print()

test_cases = [
    {
        "ingredient": "niacinamide",
        "title": "Niacinamide: a review on dermal delivery strategies and clinical evidence.",
        "abstract": "Niacinamide, an active form of vitamin B3, is recognised for its significant dermal benefits including skin brightening, anti-ageing properties and the protection of the skin barrier. Its widespread incorporation into cosmetic products is attributed to its safety profile and proven efficacy. Recently, topical niacinamide has also been explored for other pharmaceutical applications, including skin cancers."
    },
    {
        "ingredient": "parabens",
        "title": "Polycystic Ovary Syndrome and Endocrine Disruptors (Bisphenols, Parabens, and Triclosan)-A Systematic Review.",
        "abstract": "Exposure to endocrine disrupting chemicals (EDCs) can result in alterations of the female reproductive system, including polycystic ovary syndrome (PCOS). The aim of this review was to summarize the knowledge about the association of EDCs with PCOS. We evaluated the association of PCOS with bisphenols, parabens and triclosan and found significant associations."
    },
    {
        "ingredient": "niacinamide",
        "title": "Niacinamide enhances cathelicidin mediated SARS-CoV-2 membrane disruption.",
        "abstract": "Niacinamide was found to enhance the antimicrobial peptide cathelicidin's ability to disrupt the SARS-CoV-2 viral membrane, suggesting a potential therapeutic application against COVID-19."
    },
    {
        "ingredient": "parabens",
        "title": "Parabens disrupt non-canonical inflammasome activation.",
        "abstract": "Parabens are synthetic chemicals widely used as preservatives. Study of possible health hazards has been undertaken due to frequent exposure. We elucidated the effect of parabens on inflammasome induction of inflammatory responses in innate immunity."
    }
]

for tc in test_cases:
    keyword_result = classify_evidence_direction(tc["title"], tc["abstract"])
    claude_result = classify_direction_with_claude(tc["title"], tc["abstract"], tc["ingredient"])
    match = "✓" if keyword_result == claude_result else "✗ DIFFERS"
    print(f"Ingredient: {tc['ingredient']}")
    print(f"Title: {tc['title'][:60]}...")
    print(f"Keyword: {keyword_result}  |  Claude: {claude_result}  {match}")
    print()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.2/455.2 kB 8.4 MB/s eta 0:00:00
=== CLAUDE CLASSIFICATION TEST ===

Ingredient: niacinamide
Title: Niacinamide: a review on dermal delivery strategies and clin...
Keyword: SAFETY  |  Claude: SAFETY  ✓

Ingredient: parabens
Title: Polycystic Ovary Syndrome and Endocrine Disruptors (Bispheno...
Keyword: CONCERN  |  Claude: CONCERN  ✓

Ingredient: niacinamide
Title: Niacinamide enhances cathelicidin mediated SARS-CoV-2 membra...
Keyword: SAFETY  |  Claude: SAFETY  ✓

Ingredient: parabens
Title: Parabens disrupt non-canonical inflammasome activation....
Keyword: NEUTRAL  |  Claude: NEUTRAL  ✓



In [35]:
# NOURA - Cell 17: Full pipeline with Claude classification + interaction engine

def noura_scan_product_v3(product_name, raw_label, category="skincare",
                           max_ingredients=12, skip_water=True, use_claude=True):
    """
    V3 product scanner with Claude-powered classification and interaction detection.
    KB for known ingredients, Claude-classified live search for unknowns.
    """
    print(f"NOURA Product Scan: {product_name}")
    print(f"Classification: {'Claude API' if use_claude else 'Keyword matching'}")
    print("=" * 65)

    ingredients = parse_ingredient_list(raw_label)
    if skip_water:
        ingredients = [i for i in ingredients if i not in ["water", "aqua"]]

    print(f"Ingredients detected: {len(ingredients)}")
    print(f"Evaluating top {min(max_ingredients, len(ingredients))}...")
    print()

    ingredients_to_evaluate = ingredients[:max_ingredients]
    results = []
    kb_hits = 0
    live_hits = 0

    for i, ingredient in enumerate(ingredients_to_evaluate):
        kb_entry = kb_lookup(ingredient)

        if kb_entry:
            kb_hits += 1
            results.append({
                "ingredient": ingredient,
                "health_score": kb_entry["health_score"],
                "verdict": kb_entry["verdict"],
                "confidence_score": kb_entry["confidence_score"],
                "confidence_label": kb_entry["confidence_label"],
                "source": "KB",
                "studies": kb_entry["studies_reviewed"],
                "flag": (" | ".join(kb_entry["concern_flags"])) if kb_entry["concern_flags"] else None
            })
        else:
            live_hits += 1
            print(f"  [{i+1}/{len(ingredients_to_evaluate)}] Live search: {ingredient}...")
            pubmed_results = search_pubmed_normalized(ingredient, max_results=30)

            evaluated = []
            direction_count = {"SAFETY": 0, "CONCERN": 0, "NEUTRAL": 0}

            for study in pubmed_results.get("studies", []):
                abstract = study.get("abstract", "")
                source_type = classify_evidence_type(study["title"], abstract)
                direction = classify_evidence_direction_smart(
                    study["title"], abstract, ingredient, use_claude=use_claude
                )
                ev = evaluate_evidence(source_type)
                ev["study_title"] = study["title"][:80]
                ev["year"] = study["year"]
                ev["direction"] = direction
                ev["sample_size"] = study.get("sample_size", None)
                ev["abstract"] = abstract[:500]
                evaluated.append(ev)
                direction_count[direction] += 1

            score_result = calculate_direction_aware_score(evaluated)
            conf = calculate_confidence(evaluated, direction_count,
                                        pubmed_results["studies_found"])

            results.append({
                "ingredient": ingredient,
                "health_score": score_result["score"],
                "verdict": score_result["verdict"],
                "confidence_score": conf["confidence_score"],
                "confidence_label": conf["confidence_label"],
                "source": "Live+Claude" if use_claude else "Live",
                "studies": pubmed_results["studies_found"],
                "flag": score_result["flag"]
            })
            time.sleep(1)

    # Display results table
    print()
    print("=" * 65)
    print(f"NOURA PRODUCT SCAN: {product_name.upper()}")
    print("=" * 65)
    print(f"{'Ingredient':<28} {'Score':>6} {'Verdict':<16} {'Confidence':<10} {'Src'}")
    print("-" * 65)

    results_sorted = sorted(results,
                            key=lambda x: (x["health_score"] or 0),
                            reverse=True)

    for r in results_sorted:
        score_display = f"{r['health_score']}" if r['health_score'] is not None else "N/A"
        flag_marker = " ⚠" if r["flag"] else ""
        print(f"{r['ingredient']:<28} {score_display:>6} {r['verdict']:<16} "
              f"{r['confidence_label']:<10} {r['source']}{flag_marker}")

    print("=" * 65)

    # Product summary
    scored = [r for r in results if r["health_score"] is not None]
    if scored:
        avg_score = round(sum(r["health_score"] for r in scored) / len(scored), 1)
        higher_risk_count = len([r for r in results if r["verdict"] == "HIGHER RISK"])

        print()
        print(f"Product Average Score:    {avg_score}/100")
        print(f"Higher Risk Ingredients:  {higher_risk_count}")
        print(f"KB lookups (instant):     {kb_hits}")
        print(f"Live + Claude searches:   {live_hits}")

        if higher_risk_count == 0 and avg_score >= 70:
            product_verdict = "CLEAN FORMULATION"
        elif higher_risk_count >= 2 or avg_score < 40:
            product_verdict = "FORMULATION CONCERNS"
        else:
            product_verdict = "MIXED FORMULATION"

        print(f"Product Verdict:          {product_verdict}")

    # Ingredient flags
    print()
    flagged = [r for r in results if r["flag"]]
    if flagged:
        print("INGREDIENT FLAGS:")
        for r in flagged:
            print(f"  ⚠ {r['ingredient']}: {r['flag'][:120]}")

    # Interaction analysis
    all_ingredient_names = [r["ingredient"] for r in results]
    interactions = check_formula_interactions(all_ingredient_names)
    if interactions:
        # Deduplicate — remove exact duplicate descriptions
        seen_descriptions = set()
        unique_interactions = []
        for interaction in interactions:
            key = interaction["description"][:60]
            if key not in seen_descriptions:
                seen_descriptions.add(key)
                unique_interactions.append(interaction)

        print()
        display_interactions(unique_interactions, product_name)
    else:
        print()
        print("✓ No known ingredient interactions detected.")

    print("=" * 65)
    return results_sorted


# Test 1: Clean beauty serum
clean_label = """
Aqua, Aloe Barbadensis Leaf Juice, Glycerin, Niacinamide,
Sodium Hyaluronate, Ascorbic Acid, Tocopherol, Zinc Oxide,
Centella Asiatica Extract, Bakuchiol, Xanthan Gum,
Citric Acid, Sodium Benzoate
"""

noura_scan_product_v3("Clean Beauty Serum", clean_label, max_ingredients=12)

print()
print()

# Test 2: Concerning formula
concerning_label = """
Aqua, Glycerin, Retinol, Glycolic Acid, Ascorbic Acid,
Niacinamide, Sodium Benzoate, Parabens, Fragrance,
Salicylic Acid, Dimethicone, Xanthan Gum
"""

noura_scan_product_v3("Concerning Formula Test", concerning_label, max_ingredients=12)

NOURA Product Scan: Clean Beauty Serum
Classification: Claude API
Ingredients detected: 12
Evaluating top 12...

  [1/12] Live search: aloe barbadensis leaf juice...
  [8/12] Live search: centella asiatica extract...
  [9/12] Live search: bakuchiol...

NOURA PRODUCT SCAN: CLEAN BEAUTY SERUM
Ingredient                    Score Verdict          Confidence Src
-----------------------------------------------------------------
sodium hyaluronate               88 WELL SUPPORTED   VERY HIGH  KB
glycerin                         85 WELL SUPPORTED   VERY HIGH  KB
xanthan gum                      84 WELL SUPPORTED   VERY HIGH  KB
niacinamide                      82 WELL SUPPORTED   VERY HIGH  KB
citric acid                      82 WELL SUPPORTED   VERY HIGH  KB ⚠
zinc oxide                       79 WELL SUPPORTED   VERY HIGH  KB ⚠
tocopherol                       78 WELL SUPPORTED   VERY HIGH  KB ⚠
aloe barbadensis leaf juice    73.4 WELL SUPPORTED   MODERATE   Live+Claude
ascorbic acid          

[{'ingredient': 'glycerin',
  'health_score': 85,
  'verdict': 'WELL SUPPORTED',
  'confidence_score': 95,
  'confidence_label': 'VERY HIGH',
  'source': 'KB',
  'studies': 180,
  'flag': None},
 {'ingredient': 'xanthan gum',
  'health_score': 84,
  'verdict': 'WELL SUPPORTED',
  'confidence_score': 88,
  'confidence_label': 'VERY HIGH',
  'source': 'KB',
  'studies': 90,
  'flag': None},
 {'ingredient': 'niacinamide',
  'health_score': 82,
  'verdict': 'WELL SUPPORTED',
  'confidence_score': 96,
  'confidence_label': 'VERY HIGH',
  'source': 'KB',
  'studies': 200,
  'flag': None},
 {'ingredient': 'ascorbic acid',
  'health_score': 72,
  'verdict': 'WELL SUPPORTED',
  'confidence_score': 89,
  'confidence_label': 'VERY HIGH',
  'source': 'KB',
  'studies': 280,
  'flag': 'Stability concerns — degrades rapidly if poorly formulated'},
 {'ingredient': 'dimethicone',
  'health_score': 71,
  'verdict': 'WELL SUPPORTED',
  'confidence_score': 85,
  'confidence_label': 'VERY HIGH',
  'source

In [34]:
# NOURA - Cell 18: Formula interaction engine

KNOWN_INTERACTIONS = {

    # DANGEROUS combinations
    "sodium benzoate + ascorbic acid": {
        "ingredients": ["sodium benzoate", "ascorbic acid"],
        "severity": "HIGH",
        "interaction_type": "chemical reaction",
        "description": "Sodium benzoate reacts with ascorbic acid (vitamin C) in the presence of light and heat to form benzene, a known carcinogen.",
        "recommendation": "Avoid this combination. Use alternative preservatives or vitamin C derivatives that don't react."
    },

    "retinol + aha": {
        "ingredients": ["retinol", "glycolic acid", "lactic acid", "mandelic acid"],
        "severity": "MODERATE",
        "interaction_type": "pH conflict + irritation",
        "description": "AHAs work at low pH (3.0-3.5) which destabilizes retinol and increases skin irritation risk significantly.",
        "recommendation": "Use retinol and AHAs at separate times — AHAs in the morning, retinol at night. Never layer simultaneously."
    },

    "retinol + vitamin c": {
        "ingredients": ["retinol", "ascorbic acid"],
        "severity": "MODERATE",
        "interaction_type": "pH conflict",
        "description": "Vitamin C (ascorbic acid) requires acidic pH (below 3.5) while retinol is most stable at neutral pH. Combining destabilizes both actives.",
        "recommendation": "Use vitamin C in the morning routine, retinol at night."
    },

    "niacinamide + vitamin c": {
        "ingredients": ["niacinamide", "ascorbic acid"],
        "severity": "LOW",
        "interaction_type": "efficacy reduction",
        "description": "High concentrations of niacinamide and ascorbic acid can form niacin which may cause temporary skin flushing. Modern formulations at typical concentrations show minimal interaction.",
        "recommendation": "Use at typical cosmetic concentrations (under 10% each). Minimal concern in well-formulated products."
    },

    "benzoyl peroxide + retinol": {
        "ingredients": ["benzoyl peroxide", "retinol"],
        "severity": "HIGH",
        "interaction_type": "oxidation",
        "description": "Benzoyl peroxide oxidizes and degrades retinol, rendering both ineffective and potentially generating irritating byproducts.",
        "recommendation": "Never combine. Use in completely separate routines."
    },

    "aha + bha simultaneous": {
        "ingredients": ["glycolic acid", "salicylic acid"],
        "severity": "MODERATE",
        "interaction_type": "over-exfoliation",
        "description": "Combining AHA and BHA acids simultaneously increases risk of over-exfoliation, barrier damage, and sensitization.",
        "recommendation": "Alternate use — AHAs and BHAs on different days, or use pre-formulated combinations at lower concentrations."
    },

    "vitamin c + niacinamide high dose": {
        "ingredients": ["ascorbic acid", "niacinamide"],
        "severity": "LOW",
        "interaction_type": "potential flushing",
        "description": "At high concentrations both can form niacin causing temporary flushing. At standard cosmetic concentrations the risk is minimal.",
        "recommendation": "Keep both under 10% concentration. Minimal concern in typical formulations."
    },

    "copper peptides + vitamin c": {
        "ingredients": ["copper peptides", "ascorbic acid"],
        "severity": "MODERATE",
        "interaction_type": "oxidation",
        "description": "Vitamin C can oxidize copper peptides, reducing efficacy of both. Copper can also pro-oxidize vitamin C.",
        "recommendation": "Use in separate routines — copper peptides at night, vitamin C in the morning."
    },

    "parabens + fragrance": {
        "ingredients": ["parabens", "fragrance"],
        "severity": "HIGH",
        "interaction_type": "cumulative endocrine risk",
        "description": "Both parabens and certain fragrance components are known endocrine disruptors. Combined exposure increases total endocrine disruption burden.",
        "recommendation": "Avoid products containing both. Seek paraben-free, fragrance-free alternatives."
    },

    "triclosan + alcohol": {
        "ingredients": ["triclosan", "alcohol denat"],
        "severity": "MODERATE",
        "interaction_type": "absorption enhancement",
        "description": "Alcohol enhances skin penetration of triclosan, increasing systemic exposure to this endocrine disruptor.",
        "recommendation": "Avoid triclosan-containing products entirely. Use safer antimicrobial alternatives."
    },
}


def check_formula_interactions(ingredients):
    """
    Checks a list of ingredients for known dangerous interactions.
    Returns list of interactions found, sorted by severity.
    """
    ingredients_lower = [i.lower().strip() for i in ingredients]

    # Also expand via normalizer — catch vitamin c = ascorbic acid etc
    expanded = set(ingredients_lower)
    for ing in ingredients_lower:
        search_terms = normalize_ingredient(ing)
        expanded.update(search_terms)

    found_interactions = []

    for interaction_key, interaction_data in KNOWN_INTERACTIONS.items():
        involved = interaction_data["ingredients"]

        # Check how many of the interaction ingredients are present
        matches = []
        for involved_ing in involved:
            for formula_ing in expanded:
                if involved_ing in formula_ing or formula_ing in involved_ing:
                    matches.append(involved_ing)
                    break

        # Need at least 2 ingredients to have an interaction
        if len(matches) >= 2:
            found_interactions.append({
                "interaction": interaction_key,
                "severity": interaction_data["severity"],
                "type": interaction_data["interaction_type"],
                "description": interaction_data["description"],
                "recommendation": interaction_data["recommendation"],
                "matched_ingredients": matches
            })

    # Sort by severity
    severity_order = {"HIGH": 0, "MODERATE": 1, "LOW": 2}
    found_interactions.sort(key=lambda x: severity_order.get(x["severity"], 3))

    return found_interactions


def display_interactions(interactions, product_name=""):
    """Display interaction report."""
    if not interactions:
        print("✓ No known ingredient interactions detected.")
        return

    high = [i for i in interactions if i["severity"] == "HIGH"]
    moderate = [i for i in interactions if i["severity"] == "MODERATE"]
    low = [i for i in interactions if i["severity"] == "LOW"]

    print(f"NOURA INTERACTION ANALYSIS{': ' + product_name if product_name else ''}")
    print("=" * 65)
    print(f"Interactions found: {len(interactions)} "
          f"({len(high)} high | {len(moderate)} moderate | {len(low)} low)")
    print()

    for interaction in interactions:
        severity = interaction["severity"]
        marker = "🔴" if severity == "HIGH" else "🟡" if severity == "MODERATE" else "🟢"

        print(f"{marker} [{severity}] {interaction['type'].upper()}")
        print(f"   Ingredients: {' + '.join(interaction['matched_ingredients'])}")
        print(f"   {interaction['description']}")
        print(f"   → {interaction['recommendation']}")
        print()

    print("=" * 65)


# Test on our clean beauty serum
print("Testing interaction engine on Clean Beauty Serum...")
print()

clean_serum_ingredients = [
    "aloe barbadensis leaf juice", "glycerin", "niacinamide",
    "sodium hyaluronate", "ascorbic acid", "tocopherol", "zinc oxide",
    "centella asiatica extract", "bakuchiol", "xanthan gum",
    "citric acid", "sodium benzoate"
]

interactions = check_formula_interactions(clean_serum_ingredients)
display_interactions(interactions, "Clean Beauty Serum")

print()
print("Testing on concerning formula...")
print()

concerning_ingredients = [
    "aqua", "glycerin", "retinol", "glycolic acid",
    "ascorbic acid", "niacinamide", "sodium benzoate",
    "parabens", "fragrance", "salicylic acid"
]

interactions2 = check_formula_interactions(concerning_ingredients)
display_interactions(interactions2, "Concerning Formula")

Testing interaction engine on Clean Beauty Serum...

NOURA INTERACTION ANALYSIS: Clean Beauty Serum
Interactions found: 3 (1 high | 0 moderate | 2 low)

🔴 [HIGH] CHEMICAL REACTION
   Ingredients: sodium benzoate + ascorbic acid
   Sodium benzoate reacts with ascorbic acid (vitamin C) in the presence of light and heat to form benzene, a known carcinogen.
   → Avoid this combination. Use alternative preservatives or vitamin C derivatives that don't react.

🟢 [LOW] EFFICACY REDUCTION
   Ingredients: niacinamide + ascorbic acid
   High concentrations of niacinamide and ascorbic acid can form niacin which may cause temporary skin flushing. Modern formulations at typical concentrations show minimal interaction.
   → Use at typical cosmetic concentrations (under 10% each). Minimal concern in well-formulated products.

🟢 [LOW] POTENTIAL FLUSHING
   Ingredients: ascorbic acid + niacinamide
   At high concentrations both can form niacin causing temporary flushing. At standard cosmetic concentrat